**VIDEO SPLITTING**

In [ ]:
import cv2
import os

def split_video(input_video_path, output_folder, segment_duration = 20):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(input_video_path)
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frames_per_segment = int(segment_duration * fps)
    
    num_segments = int(total_frames/frames_per_segment)
    
    for i in range(num_segments):
        start_frame = i * frames_per_segment
        end_frame = start_frame + frames_per_segment
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        output_file_name = os.path.join(output_folder, f"Window_{i+1}.mp4")
        
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        fourcc = cv2.VideoWriter_fourcc(*'ffv1')
        out = cv2.VideoWriter(output_file_name, fourcc, fps, (frame_width, frame_height))
        
        for j in range(frames_per_segment):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
        
        out.release()
    
    cap.release()

input_video_path = r"D:\HRV_New\Video Database\Static\No Exercise\Oximeter\Lossless\P-1\Vijay.avi"
output_folder = r"D:\HRV_New\Video Database\Static\No Exercise\Oximeter\Lossless\P-1\P-1_Split_20_1"
split_video(input_video_path, output_folder)

**PSD - Welch function**

In [25]:
def Welch(bvps, fps, frame_count, window, minHz = 0.66, maxHz = 3.34, nfft = 512):
    """
    This function computes Welch method for spectral density estimation.

    Args:
        bvps(float32 numpy.ndarray): BVP signal as float32 Numpy.ndarray with shape [num_estimators, num_frames].
        fps (float): frames per seconds.
        minHz (float): frequency in Hz used to isolate a specific subband [minHz, maxHz] (exclusive).
        maxHz (float): frequency in Hz used to isolate a specific subband [minHz, maxHz] (exclusive).
        nfft (int): number of DFT points, specified as a positive integer.
    Returns:
        Sample frequencies as float32 numpy.ndarray, and Power spectral density or power spectrum as float32 numpy.ndarray.
    """
    fps = 30
    nperseg = 300
    noverlap = int(nperseg * 0.9)
    F, P = welch(bvps, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = nfft)
    F = F.astype(np.float32)
    P = P.astype(np.float32)
    band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
    Pfreqs = 60 * F[band]
    Power = P[:, 0, band]
    return Pfreqs, Power

**FACE DETECTION + LANDMARKING + PULSE SIGNAL EXTRACTION + FILTERING + HEART RATE PREDICTION (PSD)**

In [ ]:
# %%time
# !pip install cupy-cuda112
# !pip install plotly
# !pip install ipywidgets
# !pip install lmfit
# !pip install biosppy --user
# !pip install autorank
# !pip install tensorflow
# !pip install retinaface --user
# import pyVHR
# from pyVHR.BVP.filters import BPfilter
from retinaface import RetinaFace
import cv2
import pandas as pd
import numpy as np
import sklearn
from scipy.signal import welch, butter, filtfilt
from scipy.signal.windows import hann, kaiser

np.random.seed(0)

detector = RetinaFace(quality = "normal")

def butter_bandpass(lowcut, highcut, fps = 30, order = 4):
    nyquist = 0.5 * fps
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fps = 30, order = 4):
    b, a = butter_bandpass(lowcut, highcut, fps, order = order)
    y = filtfilt(b, a, data)
    return y

for i in range(1, 9):
    for u in range(1, 22):
        try:
                video_path = rf'D:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-{i}\Lossless\Window_{u}.avi'
                cap = cv2.VideoCapture(video_path)
                
                fc = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                
                landmark_rgb_values = {
                    'left_eye': [],
                    'right_eye': [],
                    'nose': [],
                    'mouth_left': [],
                    'mouth_right': []
                }
                
                processed_frames = 0
                
                while cap.isOpened() and processed_frames < frame_count:
                    ret, frame = cap.read()
                    if not ret:
                        break
    
                    faces = detector.predict(frame)
    
                    if faces:
                        face = faces[0] 
                        
                        regions = {
                            'left_eye': face["left_eye"],
                            'right_eye': face["right_eye"],
                            'nose': face["nose"],
                            'mouth_left': face["left_lip"],
                            'mouth_right': face["right_lip"]
                        }
    
                        for name, point in regions.items():
                            x, y = int(point[0]), int(point[1])
                            patch_size = 40
                            x1, y1 = max(0, x - patch_size), max(0, y - patch_size)
                            x2, y2 = min(frame.shape[1], x + patch_size), min(frame.shape[0], y + patch_size)
    
                            patch_region = frame[y1:y2, x1:x2]
    
                            if patch_region.size == 0:
                                print(f"Warning: Empty patch region for {name} at frame {processed_frames + 1}")
                                continue
    
                            avg_rgb = np.mean(patch_region, axis=(0, 1)).round(0)
    
                            landmark_rgb_values[name].append(avg_rgb)
    
                    processed_frames += 1
    
                cap.release()
    
                landmarks = ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']
                rgb_array = np.array([landmark_rgb_values[landmark] for landmark in landmarks])
                rgb_array = rgb_array.transpose((0, 2, 1))
                
                e1_raw = cpu_GREEN(rgb_array)
                e1_raw = e1_raw.reshape(5, 1, fc)
                e1 = butter_bandpass_filter(e1_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)
                
                e1_arrays = [arr.flatten() for arr in e1]
                e1_combined = np.stack(e1_arrays, axis=0)
                
                df = pd.DataFrame(e1_combined)
                
                headers = [f"Frame_{i+1}" for i in range(e1_combined.shape[1])]
                df.columns = headers
                                
                df.to_csv(rf'D:\HRV_New\Data\Dynamic + Static\Algorithms\GREEN\P-{i}\BVP_20_AVI\BVP_Window_{u}.csv', index = False)
                
                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e1, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr = pd.DataFrame({'BPM':bpms})
                df_hr.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr.to_csv(rf"D:\HRV_New\Data\Dynamic + Static\Algorithms\GREEN\P-{i}\BPM_20_AVI\BPM_Window_{u}.csv", index = False)
                
                e2_raw = cpu_PCA(rgb_array, component = 'all_comp')
                n_landmarks = e2_raw.size // 600 
                e2_raw = e2_raw.reshape(n_landmarks, 1, 600)
                e2 = butter_bandpass_filter(e2_raw, lowcut = 0.5, highcut = 3.7, fps = 30, order = 4)
                
                e2_arrays = [arr.flatten() for arr in e2]
                e2_combined = np.stack(e2_arrays, axis=0)
                
                df2 = pd.DataFrame(e2_combined)
                
                headers = [f"Frame_{i+1}" for i in range(e2_combined.shape[1])]
                df2.columns = headers
                                
                df2.to_csv(rf'D:\HRV_New\Data\Dynamic + Static\Algorithms\PCA\P-{i}\BVP_20_AVI\BVP_Window_{u}.csv', index = False)

                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e2, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]

                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_2 = pd.DataFrame({'BPM':bpms})
                df_hr_2.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_2.to_csv(rf"D:\HRV_New\Data\Dynamic + Static\Algorithms\PCA\P-{i}\BPM_20_AVI\BPM_Window_{u}.csv", index = False)

                e3_raw = cpu_PBV(rgb_array)
                e3_raw = e3_raw.reshape(5, 1, 600)
                e3 = butter_bandpass_filter(e3_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)

                e3_arrays = [arr.flatten() for arr in e3]
                e3_combined = np.stack(e3_arrays, axis=0)
                
                df3 = pd.DataFrame(e3_combined)
                
                headers = [f"Frame_{i+1}" for i in range(e3_combined.shape[1])]
                df3.columns = headers
                                
                df3.to_csv(rf'D:\HRV_New\Data\Dynamic + Static\Algorithms\PBV\P-{i}\BVP_20_AVI\BVP_Window_{u}.csv', index = False)
                
                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e3, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_3 = pd.DataFrame({'BPM':bpms})
                df_hr_3.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_3.to_csv(rf"D:\HRV_New\Data\Dynamic + Static\Algorithms\PBV\P-{i}\BPM_20_AVI\BPM_Window_{u}.csv", index = False)
            
                e4_raw = cpu_CHROM(rgb_array)
                e4_raw = e4_raw.reshape(5, 1, 600)
                e4 = butter_bandpass_filter(e4_raw, lowcut = 0.67, highcut = 4.00, fps = 30, order = 4)

                e4_arrays = [arr.flatten() for arr in e4]
                e4_combined = np.stack(e4_arrays, axis=0)
                
                df4 = pd.DataFrame(e4_combined)
                
                headers = [f"Frame_{i+1}" for i in range(e4_combined.shape[1])]
                df4.columns = headers
                                
                df4.to_csv(rf'D:\HRV_New\Data\Dynamic + Static\Algorithms\CHROM\P-{i}\BVP_20_AVI\BVP_Window_{u}.csv', index = False)
                
                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e4, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_4 = pd.DataFrame({'BPM':bpms})
                df_hr_4.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_4.to_csv(rf"D:\HRV_New\Data\Dynamic + Static\Algorithms\CHROM\P-{i}\BPM_20_AVI\BPM_Window_{u}.csv", index = False)
           
                e5_raw = cpu_POS(rgb_array)
                e5_raw = e5_raw.reshape(5, 1, 600)
                e5 = butter_bandpass_filter(e5_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)

                e5_arrays = [arr.flatten() for arr in e5]
                e5_combined = np.stack(e5_arrays, axis=0)
            
                df5 = pd.DataFrame(e5_combined)
                
                headers = [f"Frame_{i+1}" for i in range(e5_combined.shape[1])]
                df5.columns = headers
                                
                df5.to_csv(rf'D:\HRV_New\Data\Dynamic + Static\Algorithms\POS\P-{i}\BVP_20_AVI\BVP_Window_{u}.csv', index = False)
                
                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e5, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_5 = pd.DataFrame({'BPM':bpms})
                df_hr_5.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_5.to_csv(rf"D:\HRV_New\Data\Dynamic + Static\Algorithms\POS\P-{i}\BPM_20_AVI\BPM_Window_{u}.csv", index = False)
            
                e6_raw = cpu_LGI(rgb_array)
                e6_raw = e6_raw.reshape(5, 1, 600)
                e6 = butter_bandpass_filter(e5_raw, lowcut = 0.5, highcut = 2.50, fps = 30, order = 4)

                e6_arrays = [arr.flatten() for arr in e6]
                e6_combined = np.stack(e6_arrays, axis=0)
            
                df6 = pd.DataFrame(e6_combined)
                
                headers = [f"Frame_{i+1}" for i in range(e6_combined.shape[1])]
                df6.columns = headers
                                
                df6.to_csv(rf'D:\HRV_New\Data\Dynamic + Static\Algorithms\LGI\P-{i}\BVP_20_AVI\BVP_Window_{u}.csv', index = False)
                
                fps = 30
                nperseg = 128
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e6, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 256)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_6 = pd.DataFrame({'BPM':bpms})
                df_hr_6.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_6.to_csv(rf"D:\HRV_New\Data\Dynamic + Static\Algorithms\LGI\P-{i}\BPM_20_AVI\BPM_Window_{u}.csv", index = False)
                
                e7_raw = cpu_OMIT(rgb_array)
                e7_raw = e7_raw.reshape(5, 1, 600)
                e7 = butter_bandpass_filter(e5_raw, lowcut = 0.75, highcut = 4.00, fps = 30, order = 4)

                e7_arrays = [arr.flatten() for arr in e7]
                e7_combined = np.stack(e7_arrays, axis=0)
            
                df7 = pd.DataFrame(e7_combined)
                
                headers = [f"Frame_{i+1}" for i in range(e7_combined.shape[1])]
                df7.columns = headers
                                
                df7.to_csv(rf'D:\HRV_New\Data\Dynamic + Static\Algorithms\OMIT\P-{i}\BVP_20_AVI\BVP_Window_{u}.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = kaiser(int(nperseg), beta = 25)
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e7, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_7 = pd.DataFrame({'BPM':bpms})
                df_hr_7.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_7.to_csv(rf"D:\HRV_New\Data\Dynamic + Static\Algorithms\OMIT\P-{i}\BPM_20_AVI\BPM_Window_{u}.csv", index = False)
            
        except (ValueError, OSError) as e:
                print(e)
                pass

**PEAK DETECTION ALGORITHM**

In [104]:
from scipy.signal import find_peaks, peak_prominences, peak_widths
import numpy as np
import matplotlib.pyplot as plt

def peak_det_102(signal = [], fps = 30, win_size = 1.5):

    peaks_101 = []
    proms = []
    widths = []
    s = 0

    while win_size <= len(signal) / fps:
        x = 1.5
        t = 0

        window_signal = signal[int((win_size - x) * fps): int(win_size * fps)]

        peaks, _ = find_peaks(window_signal, height = 0)
        peaks_reverse, _ = find_peaks((window_signal * -1), height = 0)
        s += len(peaks)

        if len(peaks) == 0:
            win_size += x
            continue

        proms = peak_prominences(window_signal, peaks)[0]
        widths = peak_widths(window_signal, peaks, rel_height = 1)[0]

        if len(widths) == 0 or len(proms) == 0:
            win_size += x
            continue

        width_threshold = np.percentile(widths, 30)
        prominence_threshold = 0.25 * np.max(proms)

        peaks_2 = []

        for i in range(len(peaks)):
            if proms[i] < prominence_threshold or widths[i] < width_threshold:
                peaks_2.append(peaks[i])

        peaks_set = set(peaks)
        peaks_2_set = set(peaks_2)

        peaks_1_set = peaks_set - peaks_2_set
        peaks_1 = list(peaks_1_set)
        peaks_1.sort()

        for l in range(len(peaks_1)):
            peaks_1[l] += int((win_size - x) * fps)

        for l1 in range(len(peaks_reverse)):
            peaks_reverse[l1] += int((win_size - x) * fps)

        i = 0

        while i < len(peaks_1) - 1:
            if peaks_1[i + 1] - peaks_1[i] < 9:
                peaks_1.pop(i + 1)
            else:
                i += 1

        while t < len(peaks_1) - 1:
            start = peaks_1[t]
            end = peaks_1[t + 1]

            if any(start < value < end for value in peaks_reverse):
                t += 1
            else:
                peaks_1.pop(t + 1)

        win_size += x
        peaks_101.extend(peaks_1)

        u = 0

        while u < len(peaks_101) - 1:
            if peaks_101[u + 1] - peaks_101[u] < 9:
                peaks_101.pop(u + 1)
            else:
                u += 1
                
        while u < len(peaks_101) - 1:
            if peaks_101[u + 1] - peaks_101[u] > 45:
                peaks_101.pop(u)
            else:
                u += 1

    return peaks_101, proms, widths

**HEART RATE PREDICTION - PEAK DETECTION ALGORITHM**

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, peak_prominences, peak_widths

algos = ['GREEN', 'PCA', 'CHROM', 'LGI', 'PBV', 'POS', 'OMIT']

for algo in algos:
    for l in range(1, 7):
        folder_path = rf'D:\HRV_New\Data\Static\Algorithms\{algo}\No Exercise\P-{l}\BVP_20'
                    
        csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and 'BVP' in f]
                        
        for csv_file in csv_files:
            csv_file_path = os.path.join(folder_path, csv_file)
            df = pd.read_csv(csv_file_path)
                        
            bpms = []
            proms_1 = []
            widths_1 = []
                        
            fps = 30
            win_size = 1.5
                        
            zero_count = []
                        
            for e in range(df.shape[0]):
                y = df.iloc[e, :]
                zero_count.append(list(y).count(0))
                        
                ninety = np.percentile(zero_count, 90)
                        
            for e in range(df.shape[0]):
                y = df.iloc[e, :]
                        
                if(list(y).count(0) <= ninety):
                    t, p, w = peak_det_102(y, fps, win_size)
                    proms_1.extend(p)
                    widths_1.extend(w)
                        
                    diffs = []
                        
                    for x in range(len(t) - 1):
                        t[x] = int(t[x])
                        t[x + 1] = int(t[x + 1])
                        diff = t[x + 1] - t[x]
                        diffs.append(diff)
                        
                    if diffs:
                        avg_peak_dist = np.mean(diffs)
                        bpm = round(1800 / avg_peak_dist)
                        if(bpm > 40):
                            bpms.append(bpm)
                        else:
                            bpms.append(40)
                        
        
            output_file_path = os.path.join(rf"D:\HRV_New\Data\Static\Algorithms\{algo}\No Exercise\P-{l}\BPM_20_PDA", f"BPM_{os.path.splitext(csv_file)[0][4:]}.csv")
            df2 = pd.DataFrame({"BPM": bpms})
            df2.to_csv(output_file_path, index=False)
            print(l)

**Compiling BPM of all windows**

In [ ]:
import pandas as pd
import os

algos = ['GREEN', 'PCA', 'CHROM', 'LGI', 'PBV', 'POS', 'OMIT']

for algo in algos:
    for l in range(1, 7):
        input_folder_path = rf'D:\HRV_New\Data\Static\Algorithms\{algo}\No Exercise\P-{l}\BPM_20_PDA'
        output_file_path = rf'D:\HRV_New\Data\Static\Algorithms\{algo}\No Exercise\P-{l}\BPM_20_PDA\Avg_BPM.csv'
        
        csv_files = [f for f in os.listdir(input_folder_path) if f.endswith('.csv') and 'BPM' in f]
                        
        results = []
                        
        for csv_file in csv_files:
            csv_file_path = os.path.join(input_folder_path, csv_file)
            df = pd.read_csv(csv_file_path)
                        
            avg_bpm = round(df['BPM'].mean(), 0)
                        
            video_name = csv_file.replace('BPM_Window_', '').replace('.csv', '')
            
            video_num = int(video_name)

            results.append({'Video': video_name, 'Algo': avg_bpm, 'Video_Num': video_num})
        
        results_df = pd.DataFrame(results)

        results_df = results_df.sort_values(by = 'Video_Num')
        
        results_df = results_df.drop(columns = ['Video_Num'])
                        
        results_df.to_csv(output_file_path, index=False)
        print(l)

**AVERAGE BPM COMPILATION OF ALL ALGORITHMS**

In [130]:
import os
import pandas as pd

base_dir = r"D:\HRV_New\Data\Static\Algorithms"

combined_df = pd.DataFrame()

for l in range(1,7):
    for root, dirs, files in os.walk(base_dir):
        if os.path.basename(root) == f"P-{l}":
            bpm_subdir = os.path.join(root, "BPM_20_PDA")
            
            csv_path = os.path.join(bpm_subdir, "Avg_BPM.csv").strip()
            csv_path = os.path.normpath(csv_path)
    
            if os.path.exists(csv_path):
                try:
                    df = pd.read_csv(csv_path)
                    
                    if "Algo" in df.columns:
                        parent_name = os.path.basename(os.path.dirname(os.path.dirname(root)))
                        combined_df[f"Algo_{parent_name}"] = df["Algo"]
                    else:
                        print(f"'Algo' column not found in {csv_path}")
                except Exception as e:
                    print(f"Error reading {csv_path}: {e}")
            else:
                print(f"File does not exist: {csv_path}")
    
    desired_order = [
        "Algo_GREEN", "Algo_PCA", "Algo_PBV", "Algo_CHROM", "Algo_POS", "Algo_LGI", "Algo_OMIT",
        "Algo_PhysNet", "Algo_RhythmNet", "Algo_EfficientPhys", "Algo_JAMSNet",
        "Algo_RTrPPG", "Algo_LSTC-rPPG", "Algo_BigSmall"
    ]
    
    combined_df = combined_df.reindex(columns = desired_order)
    
    output_path = rf"D:\HRV_New\Data\Static\Algorithms\P-{l}_Avg_BPM.csv"
    combined_df.to_csv(output_path, index=False)

**ALGORITHMS**

In [7]:
def cpu_GREEN(signal):
    """
    GREEN method on CPU using Numpy

    Verkruysse, W., Svaasand, L. O., & Nelson, J. S. (2008). Remote plethysmographic imaging using ambient light. Optics express, 16(26), 21434-21445.
    """
    return signal[:,1,:]

In [8]:
from sklearn.decomposition import PCA

def cpu_PCA(signal,**kargs):
    """
    PCA method on CPU using Numpy.

    The dictionary parameters are {'component':str}. Where 'component' can be 'second_comp' or 'all_comp'.

    Lewandowska, M., Rumiński, J., Kocejko, T., & Nowak, J. (2011, September). Measuring pulse rate with a webcam—a non-contact method for evaluating cardiac activity. In 2011 federated conference on computer science and information systems (FedCSIS) (pp. 405-410). IEEE.
    """
    bvp = []
    for i in range(signal.shape[0]):
        X = signal[i]
        pca = PCA(n_components=3)
        pca.fit(X)

        # selector
        if kargs['component']=='all_comp':
            bvp.append(pca.components_[0] * pca.explained_variance_[0])
            bvp.append(pca.components_[1] * pca.explained_variance_[1])
        elif kargs['component']=='second_comp':
            bvp.append(pca.components_[1] * pca.explained_variance_[1])
    bvp = np.array(bvp)
    return bvp

In [9]:
def cpu_CHROM(signal):
    """
    CHROM method on CPU using Numpy.

    De Haan, G., & Jeanne, V. (2013). Robust pulse rate from chrominance-based rPPG. 
    IEEE Transactions on Biomedical Engineering, 60(10), 2878-2886.
    """
    X = signal
    Xcomp = 3*X[:, 0] - 2*X[:, 1]
    Ycomp = (1.5*X[:, 0])+X[:, 1]-(1.5*X[:, 2])
    sX = np.std(Xcomp, axis=1)
    sY = np.std(Ycomp, axis=1)
    alpha = (sX/sY).reshape(-1, 1)
    alpha = np.repeat(alpha, Xcomp.shape[1], 1)
    bvp = Xcomp - np.multiply(alpha, Ycomp)
    return bvp

In [10]:
def cpu_PBV(signal):
    """
    PBV method on CPU using Numpy.

    De Haan, G., & Van Leest, A. (2014). Improved motion robustness of remote-PPG by using the blood volume pulse signature. Physiological measurement, 35(9), 1913.
    """
    sig_mean = np.mean(signal, axis = 2)

    signal_norm_r = signal[:,0,:] / np.expand_dims(sig_mean[:,0],axis=1) - 1
    signal_norm_g = signal[:,1,:] / np.expand_dims(sig_mean[:,1],axis=1) - 1
    signal_norm_b = signal[:,2,:] / np.expand_dims(sig_mean[:,2],axis=1) - 1

    pbv_n = np.array([np.std(signal_norm_r, axis = 1), np.std(signal_norm_g, axis = 1), np.std(signal_norm_b, axis = 1)])
    pbv_d = np.sqrt(np.var(signal_norm_r, axis = 1) + np.var(signal_norm_g, axis = 1) + np.var(signal_norm_b, axis = 1))
    pbv = pbv_n / pbv_d

    C = np.swapaxes(np.array([signal_norm_r, signal_norm_g, signal_norm_b]), 0, 1)
    Ct = np.swapaxes(np.swapaxes(np.transpose(C),0,2),1,2)
    Q = np.matmul(C, Ct)
    W = np.linalg.solve(Q,np.swapaxes(pbv,0,1))

    A = np.matmul(Ct, np.expand_dims(W,axis = 2))
    B =  np.matmul(np.swapaxes(np.expand_dims(pbv.T,axis=2),1,2),np.expand_dims(W,axis = 2))
    bvp = A / B
    return bvp.squeeze(axis=2)

In [11]:
import numpy as np

def cpu_POS(signal, **kargs):
    """
    POS method on GPU using Cupy.

    The dictionary parameters are: {'fps':float}.

    Wang, W., den Brinker, A. C., Stuijk, S., & de Haan, G. (2016). Algorithmic principles of remote PPG. IEEE Transactions on Biomedical Engineering, 64(7), 1479-1491. 
    """
    # Run the POS algorithm on the RGB color signal c with sliding window length wlen
    # Recommended value for wlen is 32 for a 20 fps camera (1.6 s)
    eps = 10**-9
    X = signal
    fps = 30
    e, c, f = X.shape # e = estimators, c = 3 rgb channels, f = frames
    w = int(1.5 * fps)   # window length
    # stack e times fixed mat P
    P = np.array([[0, 1, -1], [-2, 1, 1]])
    Q = np.stack([P for _ in range(e)], axis=0)

    # Initialize (1)
    H = np.zeros((e, f))
    for n in np.arange(w, f, 3):
        # Start index of sliding window (4)
        m = n - w + 2
        X = np.array(X)
        # Temporal normalization (5)
        Cn = X[:, :, m:(n + 2)]
        M = 1.0 / (np.mean(Cn, axis = 2)+eps)
        M = np.expand_dims(M, axis=2)  # shape [e, c, w]
        Cn = np.multiply(M, Cn)

        # Projection (6)
        S = np.dot(Q, Cn)
        S = S[0, :, :, :]
        S = np.swapaxes(S, 0, 1)    # remove 3-th dim

        # Tuning (7)
        S1 = S[:, 0, :]
        S2 = S[:, 1, :]
        alpha = np.std(S1, axis=1) / (eps + np.std(S2, axis=1))
        alpha = np.expand_dims(alpha, axis=1)
        Hn = np.add(S1, alpha * S2)
        Hnm = Hn - np.expand_dims(np.mean(Hn, axis=1), axis=1)
        # Overlap-adding (8)
        H[:, m:(n + 2)] = np.add(H[:, m: (n + 2)], Hnm)

    return H

In [12]:
def cpu_LGI(signal):
    """
    LGI method on CPU using Numpy.

    Pilz, C. S., Zaunseder, S., Krajewski, J., & Blazek, V. (2018). Local group invariance for heart rate estimation from face videos in the wild. In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition Workshops (pp. 1254-1262).
    """
    X = signal
    U, _, _ = np.linalg.svd(X)
    S = U[:, :, 0]
    S = np.expand_dims(S, 2)
    sst = np.matmul(S, np.swapaxes(S, 1, 2))
    p = np.tile(np.identity(3), (S.shape[0], 1, 1))
    P = p - sst
    Y = np.matmul(P, X)
    bvp = Y[:, 1, :]
    return bvp

In [13]:
def cpu_OMIT(signal):
    """
    OMIT method on CPU using Numpy.

    Álvarez Casado, C., Bordallo López, M. (2022). Face2PPG: An unsupervised pipeline for blood volume pulse extraction from faces. arXiv (eprint 2202.04101).
    """

    bvp = []
    for i in range(signal.shape[0]):
        X = signal[i]
        Q, R = np.linalg.qr(X)
        S = Q[:, 0].reshape(1, -1)
        P = np.identity(3) - np.matmul(S.T, S)
        Y = np.dot(P, X)
        bvp.append(Y[1, :])
    bvp = np.array(bvp)
    return bvp

**GROUND TRUTH CALCULATIONS - DYNAMIC + STATIC**

In [30]:
import pandas as pd

for l in range(4, 9):
        file_path = rf'D:\HRV_New\Data\Dynamic + Static\Ground Truth\P-{l}.csv'
        df = pd.read_csv(file_path, skiprows = 5)
            
        df_filtered = df[df['Time(s)'] >= 1]
        
        window_numbers = []
        gt_avg = []
        gt_ranges = []
        
        for i in range(0, len(df_filtered), 20):
            window_data = df_filtered.iloc[i:i+20]
            hr_values = window_data['HR(BPM)']
            
            avg_hr = hr_values.mean()
            min_hr = hr_values.min()
            max_hr = hr_values.max()
            
            window_numbers.append(i//20 + 1)
            gt_avg.append(round(avg_hr))
            gt_ranges.append(f"{min_hr} - {max_hr}")
        
        output_df = pd.DataFrame({
            'Window': window_numbers,
            'GT': gt_ranges,
            'GT_Avg': gt_avg
        })
        
        output_csv_path = rf"D:\HRV_New\Data\Dynamic + Static\Ground Truth\Stats\P-{l}.csv" 
        output_df.to_csv(output_csv_path, index=False)

**GROUND TRUTH CALCULATIONS - DYNAMIC**

In [31]:
import pandas as pd

for l in range(7, 8):
        file_path = rf'D:\HRV_New\Data\Dynamic\Ergobike\Ground Truth\HRV_Dynamic_GT_Database - P-7 (Rohit).csv'
        df = pd.read_csv(file_path, skiprows = 5)
            
        df_filtered = df[df['Time(s)'] >= 1]
        
        window_numbers = []
        gt_avg = []
        gt_ranges = []
        
        for i in range(0, len(df_filtered), 20):
            window_data = df_filtered.iloc[i:i+20]
            hr_values = window_data['HR(BPM)']
            
            avg_hr = hr_values.mean()
            min_hr = hr_values.min()
            max_hr = hr_values.max()
            
            window_numbers.append(i//20 + 1)
            gt_avg.append(round(avg_hr))
            gt_ranges.append(f"{min_hr} - {max_hr}")
        
        output_df = pd.DataFrame({
            'Window': window_numbers,
            'GT': gt_ranges,
            'GT_Avg': gt_avg
        })
        
        output_csv_path = rf"D:\HRV_New\Data\Dynamic\Ergobike\Ground Truth\Stats\20 seconds\P-{l}.csv" 
        output_df.to_csv(output_csv_path, index=False)

**Error Calculations - Dynamic**

In [137]:
import pandas as pd
import numpy as np

for l in range(1, 7):
        file_path = rf'D:\HRV_New\Data\Static\Ground Truth\02_09_24\HRV_Static_Benchmarking_Data_PDA - P-{l}.csv' 
        df = pd.read_csv(file_path, header = 2)
        
        columns_of_interest = ['GT_Avg','Algo_GREEN', 'Algo_PCA', 'Algo_PBV', 'Algo_CHROM', 'Algo_POS', 
                               'Algo_LGI', 'Algo_OMIT', 'Algo_PhysNet(NN)', 'Algo_RhythmNet(NN)', 
                               'Algo_EfficientPhys(NN)', 'Algo_JAMSNet(NN)', 'Algo_RTrPPG(NN)', 
                               'Algo_LSTCrPPG(NN)', 'Algo_BigSmall(NN)']
        
        df_filtered = df[columns_of_interest]
        
        df_filtered = df_filtered.apply(pd.to_numeric, errors = 'coerce')
        
        # df_filtered = df_filtered.dropna()
        
        mae_values = []
        rmse_values = []
        
        for column in columns_of_interest[1:]:
            mae = np.mean(np.abs(df_filtered[column] - df_filtered['GT_Avg']))
            rmse = np.sqrt(np.mean((df_filtered[column] - df_filtered['GT_Avg'])**2))
            mae_values.append(mae)
            rmse_values.append(rmse)
        
        average_mae = np.mean(mae_values)
        average_rmse = np.mean(rmse_values)
        
        results = pd.DataFrame({'MAE': mae_values, 'RMSE': rmse_values})
        results = results.transpose()
        
        results.to_csv(rf'D:\HRV_New\Data\Static\Errors\P-{l}\Errors_Average_PDA.csv', index=False)

**ERROR CALCULATIONS - DYNAMIC + STATIC**

In [102]:
import pandas as pd
import numpy as np

for l in range(5, 6):
    file_path = rf'D:\HRV_New\Data\Dynamic + Static\Ground Truth\HRV_Data_Final - P-{l}.csv' 
    df = pd.read_csv(file_path, header = 2)

    df.columns = df.columns.str.strip()
    
    columns_of_interest = ['GT_Avg', 'Movement', 'Algo_GREEN', 'Algo_PCA', 'Algo_PBV', 'Algo_CHROM', 'Algo_POS', 
                           'Algo_LGI', 'Algo_OMIT', 'Algo_PhysNet(NN)', 'Algo_RhythmNet(NN)', 
                           'Algo_EfficientPhys(NN)', 'Algo_JAMSNet(NN)', 'Algo_RTrPPG(NN)', 
                           'Algo_LSTCrPPG(NN)', 'Algo_BigSmall(NN)']
    
    df_filtered = df[columns_of_interest]
    
    df_filtered = df_filtered.apply(pd.to_numeric, errors = 'coerce', axis=1)

    df_filtered["Movement"] = df["Movement"]
    
    # df_filtered = df_filtered.dropna()

    df_yes_movement = df_filtered[df_filtered['Movement'] == 'Yes']
    df_no_movement = df_filtered[df_filtered['Movement'] == 'No']
    
    def calculate_errors(df_subset):
        mae_values = []
        rmse_values = []
        
        for column in columns_of_interest[2:]:
            mae = np.mean(np.abs(df_subset[column] - df_subset['GT_Avg']))
            rmse = np.sqrt(np.mean((df_subset[column] - df_subset['GT_Avg'])**2))
            mae_values.append(mae)
            rmse_values.append(rmse)
        
        return mae_values, rmse_values

    mae_values_yes, rmse_values_yes = calculate_errors(df_yes_movement)
    mae_values_no, rmse_values_no = calculate_errors(df_no_movement)

    results_yes = pd.DataFrame({'MAE_Dynamic': mae_values_yes, 'RMSE_Dynamic': rmse_values_yes}, index = columns_of_interest[2:])
    results_no = pd.DataFrame({'MAE_Static': mae_values_no, 'RMSE_Static': rmse_values_no}, index = columns_of_interest[2:])
    
    combined_results = pd.concat([results_yes, results_no], axis=1).T
    
    output_file_path = rf'D:\HRV_New\Data\Dynamic + Static\Errors\P-{l}\Errors_Average_PDA.csv'
    combined_results.to_csv(output_file_path)

**DENOISING ATTEMPTS**

In [18]:
# !pip install pandas
# !pip install PyWavelets
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, find_peaks, welch
from scipy.signal.windows import hann
import pywt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_bandpass_filter(signal, lowcut = 0.75, highcut = 3.33, fs=30):
    b, a = butter_bandpass(lowcut, highcut, fs, order=5)
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

def wavelet_denoise(signal, wavelet='db4', level = 2):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745
    threshold = sigma * np.sqrt(2 * np.log(len(signal)))
    denoised_coeffs = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
    return pywt.waverec(denoised_coeffs, wavelet)

def kalman_filter(z, Q=1e-7, R=1e-3):
    n = len(z)
    xhat = np.zeros(n)
    P = np.zeros(n)
    xhat[0] = z[0]
    P[0] = 1.0

    for k in range(1, n):
        xhat[k] = xhat[k-1]
        P[k] = P[k-1] + Q

        K = P[k] / (P[k] + R)
        xhat[k] = xhat[k] + K * (z[k] - xhat[k])
        P[k] = (1 - K) * P[k]

    return xhat

def noise_reduction_pipeline(signal, fs = 30):
    filtered_signal = apply_bandpass_filter(signal, lowcut = 0.75, highcut = 3.33, fs=fs)
    denoised_signal = wavelet_denoise(filtered_signal, wavelet='db4', level = 2)
    smoothed_signal = kalman_filter(denoised_signal)
    return smoothed_signal

def compute_snr(signal, noise):
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def compute_rmse(original_signal, cleaned_signal):
    return np.sqrt(np.mean((original_signal - cleaned_signal) ** 2))

def compute_bpm(signal, fs = 30):
    peaks, _ = find_peaks(signal, distance = fs * 0.3)
    peak_intervals = np.diff(peaks) / fs
    bpm = 60 / np.mean(peak_intervals)
    return bpm
    
for l in range(1, 10):
    df = pd.read_csv(rf"D:\BIDMC dataset\bidmc-ppg-and-respiration-dataset-1.0.0\bidmc_csv\bidmc_0{l}_Signals.csv")
    total_samples = int(len(df[df.columns[2]])/125)
    print(total_samples)
    # window_size = 64 * 8 
    # window_shift = 64 * 2
    
    hrs = []
    hrs_peak_det = []
    
    for i in range(total_samples):   
        rppg_signal = df[df.columns[2]][i  * 125:(i + 1) * 125]
        # print(rppg_signal.shape)
        cleaned_signal = noise_reduction_pipeline(rppg_signal)[:125]
        # print(cleaned_signal.shape)
        
        noise_signal = rppg_signal - cleaned_signal
        snr = compute_snr(rppg_signal, noise_signal)
        rmse = compute_rmse(rppg_signal, cleaned_signal)
        bpm_cleaned = compute_bpm(cleaned_signal, fs = 30)
        bpm_original = compute_bpm(rppg_signal, fs = 30)
    
        fps = 125
        nperseg = 1
        noverlap = int(nperseg * 0.9)
        window = hann(nperseg)
        minHz = 0.67
        maxHz = 3.33
        F, P = welch(rppg_signal, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window, nfft = 512)
        F = F.astype(np.float32)
        P = P.astype(np.float32)
        band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
        Pfreqs = 60 * F[band]
        Power = P[band]
        Pmax = np.argmax(Power, axis = 0)
        bpm = (np.round(Pfreqs[Pmax.squeeze()]))
        hrs.append(bpm)
        hrs_peak_det.append(bpm_cleaned)
    
    df2 = pd.DataFrame({"Heart Rate(PSD)":hrs, "Heart Rate(Peak Det)":np.round(hrs_peak_det)})
    df2.to_csv(r"D:\BIDMC dataset\bidmc-ppg-and-respiration-dataset-1.0.0\bidmc_csv\HR_PPG_{l}.csv", index = False)

480
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(1

D:\Anaconda\envs\pyvhr\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Anaconda\envs\pyvhr\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)
(125,)

In [ ]:
# !pip install pandas
# !pip install PyWavelets
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, find_peaks, welch
from scipy.signal.windows import hann
import pywt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_bandpass_filter(signal, lowcut = 0.75, highcut = 3.33, fs=30):
    b, a = butter_bandpass(lowcut, highcut, fs, order=5)
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

def wavelet_denoise(signal, wavelet='db4', level = 2):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745
    threshold = sigma * np.sqrt(2 * np.log(len(signal)))
    denoised_coeffs = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
    return pywt.waverec(denoised_coeffs, wavelet)

def kalman_filter(z, Q=1e-7, R=1e-3):
    n = len(z)
    xhat = np.zeros(n)
    P = np.zeros(n)
    xhat[0] = z[0]
    P[0] = 1.0

    for k in range(1, n):
        xhat[k] = xhat[k-1]
        P[k] = P[k-1] + Q

        K = P[k] / (P[k] + R)
        xhat[k] = xhat[k] + K * (z[k] - xhat[k])
        P[k] = (1 - K) * P[k]

    return xhat

def noise_reduction_pipeline(signal, fs = 30):
    filtered_signal = apply_bandpass_filter(signal, lowcut = 0.75, highcut = 3.33, fs=fs)
    denoised_signal = wavelet_denoise(filtered_signal, wavelet='db4', level = 2)
    smoothed_signal = kalman_filter(denoised_signal)
    return smoothed_signal

def compute_snr(signal, noise):
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def compute_rmse(original_signal, cleaned_signal):
    return np.sqrt(np.mean((original_signal - cleaned_signal) ** 2))

def compute_bpm(signal, fs = 30):
    peaks, _ = find_peaks(signal, distance = fs * 0.3)
    peak_intervals = np.diff(peaks) / fs
    bpm = 60 / np.mean(peak_intervals)
    return bpm
    
for l in range(1, 10):
    df = pd.read_csv(rf"D:\BIDMC dataset\bidmc-ppg-and-respiration-dataset-1.0.0\bidmc_csv\bidmc_0{l}_Signals.csv")
    total_samples = int(len(df[df.columns[2]])/125)
    print(total_samples)
    # window_size = 64 * 8 
    # window_shift = 64 * 2
    
    hrs = []
    hrs_peak_det = []
    
    for i in range(total_samples):   
        rppg_signal = df[df.columns[2]][i  * 125:(i + 1) * 125]
        print(rppg_signal.shape)
        cleaned_signal = noise_reduction_pipeline(rppg_signal)[:125]
        print(cleaned_signal.shape)
        
        noise_signal = rppg_signal - cleaned_signal
        snr = compute_snr(rppg_signal, noise_signal)
        rmse = compute_rmse(rppg_signal, cleaned_signal)
        bpm_cleaned = compute_bpm(cleaned_signal, fs = 30)
        bpm_original = compute_bpm(rppg_signal, fs = 30)
    
        fps = 125
        nperseg = 1
        noverlap = int(nperseg * 0.9)
        window = hann(nperseg)
        minHz = 0.67
        maxHz = 3.33
        F, P = welch(cleaned_signal, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window, nfft = 512)
        F = F.astype(np.float32)
        P = P.astype(np.float32)
        band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
        Pfreqs = 60 * F[band]
        Power = P[band]
        Pmax = np.argmax(Power, axis = 0)
        bpm = (np.round(Pfreqs[Pmax.squeeze()]))
        hrs.append(bpm)
        hrs_peak_det.append(bpm_cleaned)
    
    df2 = pd.DataFrame({"Heart Rate(PSD)":hrs, "Heart Rate(Peak Det)":np.round(hrs_peak_det)})
    df2.to_csv(r"D:\BIDMC dataset\bidmc-ppg-and-respiration-dataset-1.0.0\bidmc_csv\HR_PPG_{l}.csv", index = False)

**MP4 TO AVI**

In [1]:
import os
import subprocess

ffmpeg_path = r"D:\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe"

for l in range(3, 8):
    input_dir = rf"D:\HRV_New\Video Database\Dynamic\Face Videos\P-{l}\P-{l}_Split_20"
    output_dir = os.path.join(input_dir, "Lossless")
    
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.endswith(".mp4"):
            input_file = os.path.join(input_dir, filename)
            output_file = os.path.join(output_dir, filename.replace(".mp4", ".avi"))
    
            command = [
                ffmpeg_path,
                '-i', input_file,         
                '-c:v', 'rawvideo',        
                '-pix_fmt', 'yuv420p',     
                '-r', '30',                
                '-an',                     
                output_file                
            ]

            subprocess.run(command, check=True)

**UBFC - GT BVP**

In [2]:
import pandas as pd

for l in range(1, 43):
    input_file = rf'D:\UBFC\P-{l}\ground_truth.txt'
    
    values = []
    
    with open(input_file, 'r') as file:
        for line in file:
            values.extend(line.strip().split())
    
    total_values = len(values)
    split_size = total_values // 3
    
    bvp_values = values[:split_size]
    hr_values = values[split_size: 2 * split_size]
    time_values = values[2 * split_size:]
    
    df = pd.DataFrame({
        'BVP': bvp_values,
        'HR': hr_values,
        'Time': time_values
    })
    
    output_file = rf'D:\UBFC\P-{l}\BVP.csv'
    df.to_csv(output_file, index=False)

**UBFC - FACE DETECTION + LANDMARKING + PULSE SIGNAL EXTRACTION + FILTERING + HEART RATE PREDICTION (PSD)**

In [ ]:
from retinaface import RetinaFace
import cv2
import pandas as pd
import numpy as np
import sklearn
from scipy.signal import welch, butter, filtfilt
from scipy.signal.windows import hann, kaiser

np.random.seed(0)

detector = RetinaFace(quality = "normal")

def butter_bandpass(lowcut, highcut, fps = 30, order = 4):
    nyquist = 0.5 * fps
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fps = 30, order = 4):
    b, a = butter_bandpass(lowcut, highcut, fps, order = order)
    y = filtfilt(b, a, data)
    return y

for i in range(1, 43):
        print(i)
        try:
                video_path = rf'D:\UBFC\P-{i}\vid.AVI'
                cap = cv2.VideoCapture(video_path)
                
                fc = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                
                landmark_rgb_values = {
                    'left_eye': [],
                    'right_eye': [],
                    'nose': [],
                    'mouth_left': [],
                    'mouth_right': []
                }
                
                processed_frames = 0
                
                while cap.isOpened() and processed_frames < fc:
                    ret, frame = cap.read()
                    if not ret:
                        break
    
                    faces = detector.predict(frame)
    
                    if faces:
                        face = faces[0] 
                        
                        regions = {
                            'left_eye': face["left_eye"],
                            'right_eye': face["right_eye"],
                            'nose': face["nose"],
                            'mouth_left': face["left_lip"],
                            'mouth_right': face["right_lip"]
                        }
    
                        for name, point in regions.items():
                            x, y = int(point[0]), int(point[1])
                            patch_size = 40
                            x1, y1 = max(0, x - patch_size), max(0, y - patch_size)
                            x2, y2 = min(frame.shape[1], x + patch_size), min(frame.shape[0], y + patch_size)
    
                            patch_region = frame[y1:y2, x1:x2]
    
                            if patch_region.size == 0:
                                print(f"Warning: Empty patch region for {name} at frame {processed_frames + 1}")
                                continue
    
                            avg_rgb = np.mean(patch_region, axis=(0, 1)).round(0)
    
                            landmark_rgb_values[name].append(avg_rgb)
    
                    processed_frames += 1
    
                cap.release()
    
                landmarks = ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']
                rgb_array = np.array([landmark_rgb_values[landmark] for landmark in landmarks])
                rgb_array = rgb_array.transpose((0, 2, 1))
                
                # rgb_array_2 = rgb_array - np.mean(rgb_array, axis = 2, keepdims = True)
                # rgb_array_3 = butter_bandpass_filter(rgb_array, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)
                e1_raw = cpu_GREEN(rgb_array)
                e1_raw = e1_raw.reshape(5, 1, fc)
                e1 = butter_bandpass_filter(e1_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)
                
                e1_arrays = [arr.flatten() for arr in e1]
                e1_combined = np.stack(e1_arrays, axis=0)
                
                df = pd.DataFrame(e1_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e1_combined.shape[1])]
                df.columns = headers
                                
                df.to_csv(rf'D:\UBFC\P-{i}\GREEN\BVP_AVI\BVP_2.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e1, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr = pd.DataFrame({'BPM':bpms})
                df_hr.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr.to_csv(rf"D:\UBFC\P-{i}\GREEN\BPM_AVI\BPM_2.csv", index = False)
                
                # rgb_array_5 = butter_bandpass_filter(rgb_array, lowcut = 0.5, highcut = 3.7, fps = 30, order = 4)
                e2_raw = cpu_PCA(rgb_array, component = 'all_comp')
                n_landmarks = e2_raw.size // fc 
                e2_raw = e2_raw.reshape(n_landmarks, 1, fc)
                e2 = butter_bandpass_filter(e2_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)
                
                e2_arrays = [arr.flatten() for arr in e2]
                e2_combined = np.stack(e2_arrays, axis=0)
                
                df2 = pd.DataFrame(e2_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e2_combined.shape[1])]
                df2.columns = headers
                                
                df2.to_csv(rf'D:\UBFC\P-{i}\PCA\BVP_AVI\BVP_2.csv', index = False)

                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e2, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]

                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_2 = pd.DataFrame({'BPM':bpms})
                df_hr_2.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_2.to_csv(rf"D:\UBFC\P-{i}\PCA\BPM_AVI\BPM_2.csv", index = False)

                # rgb_array_bp = butter_bandpass_filter(rgb_array, lowcut = 0.5, highcut = 3.5, fps = 30, order = 4)
                e3_raw = cpu_PBV(rgb_array)
                e3_raw = e3_raw.reshape(5, 1, fc)
                e3 = butter_bandpass_filter(e3_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)

                e3_arrays = [arr.flatten() for arr in e3]
                e3_combined = np.stack(e3_arrays, axis = 0)
                
                df3 = pd.DataFrame(e3_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e3_combined.shape[1])]
                df3.columns = headers
                                
                df3.to_csv(rf'D:\UBFC\P-{i}\PBV\BVP_AVI\BVP_2.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e3, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr = pd.DataFrame({'BPM':bpms})
                df_hr.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_3.to_csv(rf"D:\UBFC\P-{i}\PBV\BPM_AVI\BPM_2.csv", index = False)
            
                # rgb_array_6 = rgb_array/np.mean(rgb_array, axis = 2, keepdims = True)
                # rgb_array_7 = butter_bandpass_filter_PCA(rgb_array_6, lowcut = 0.5, highcut = 3.7, fps = 30, order = 4)
                e4_raw = cpu_CHROM(rgb_array)
                e4_raw = e4_raw.reshape(5, 1, fc)
                e4 = butter_bandpass_filter(e4_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)

                e4_arrays = [arr.flatten() for arr in e4]
                e4_combined = np.stack(e4_arrays, axis=0)
                
                df4 = pd.DataFrame(e4_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e4_combined.shape[1])]
                df4.columns = headers
                                
                df4.to_csv(rf'D:\UBFC\P-{i}\CHROM\BVP_AVI\BVP_2.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e4, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr = pd.DataFrame({'BPM':bpms})
                df_hr.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_4.to_csv(rf"D:\UBFC\P-{i}\CHROM\BPM_AVI\BPM_2.csv", index = False)
           
                e5_raw = cpu_POS(rgb_array)
                e5_raw = e5_raw.reshape(5, 1, fc)
                e5 = butter_bandpass_filter(e5_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)

                # e5_arrays = [arr.flatten() for arr in e5]
                # e5_combined = np.stack(e5_arrays, axis=0)
            
                # df5 = pd.DataFrame(e5_combined)
                
                # headers = [f"Frame_{j+1}" for j in range(e5_combined.shape[1])]
                # df5.columns = headers
                                
                # df5.to_csv(rf'D:\UBFC\P-{i}\POS\BVP_AVI\BVP.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e5, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr = pd.DataFrame({'BPM':bpms})
                df_hr.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_5.to_csv(rf"D:\UBFC\P-{i}\POS\BPM_AVI\BPM_2.csv", index = False)
            
                e6_raw = cpu_LGI(rgb_array)
                e6_raw = e6_raw.reshape(5, 1, fc)
                e6 = butter_bandpass_filter(e6_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)

                e6_arrays = [arr.flatten() for arr in e6]
                e6_combined = np.stack(e6_arrays, axis = 0)
            
                df6 = pd.DataFrame(e6_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e6_combined.shape[1])]
                df6.columns = headers
                                
                df6.to_csv(rf'D:\UBFC\P-{i}\LGI\BVP_AVI\BVP_2.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e6, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr = pd.DataFrame({'BPM':bpms})
                df_hr.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_6.to_csv(rf"D:\UBFC\P-{i}\LGI\BPM_AVI\BPM_2.csv", index = False)
                
                # rgb_array_9 = butter_bandpass_filter(rgb_array, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4, alpha = 25)
                e7_raw = cpu_OMIT(rgb_array)
                e7_raw = e7_raw.reshape(5, 1, fc)
                e7 = butter_bandpass_filter(e7_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)

                e7_arrays = [arr.flatten() for arr in e7]
                e7_combined = np.stack(e7_arrays, axis=0)
            
                df7 = pd.DataFrame(e7_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e7_combined.shape[1])]
                df7.columns = headers
                                
                df7.to_csv(rf'D:\UBFC\P-{i}\OMIT\BVP_AVI\BVP_2.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e7, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr = pd.DataFrame({'BPM':bpms})
                df_hr.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_7.to_csv(rf"D:\UBFC\P-{i}\OMIT\BPM_AVI\BPM_2.csv", index = False)
            
        except (ValueError, OSError) as e:
                print(e)
                pass

In [ ]:
from retinaface import RetinaFace
import cv2
import pandas as pd
import numpy as np
import sklearn
from scipy.signal import welch, butter, filtfilt
from scipy.signal.windows import hann, kaiser, hamming, kaiser_bessel_derived

np.random.seed(0)

detector = RetinaFace(quality = "normal")

def butter_bandpass(lowcut, highcut, fps = 30, order = 4):
    nyquist = 0.5 * fps
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fps = 30, order = 4):
    b, a = butter_bandpass(lowcut, highcut, fps, order = order)
    y = filtfilt(b, a, data)
    return y

def butter_bandpass_filter_OMIT(data, lowcut, highcut, fps=30, order=4, alpha = 25.0):
    window = kaiser(5, beta = 25)
    window = window.reshape(-1, 1, 1)
    
    windowed_data = data * window
    
    b, a = butter_bandpass(lowcut, highcut, fps, order=order)
    
    filtered_data = filtfilt(b, a, windowed_data)
    return filtered_data

for i in range(1, 2):
        print(i)
        try:
                video_path = rf'D:\UBFC\P-{i}\vid.AVI'
                cap = cv2.VideoCapture(video_path)
                
                fc = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                
                landmark_rgb_values = {
                    'left_eye': [],
                    'right_eye': [],
                    'nose': [],
                    'mouth_left': [],
                    'mouth_right': []
                }
                
                processed_frames = 0
                
                while cap.isOpened() and processed_frames < fc:
                    ret, frame = cap.read()
                    if not ret:
                        break
    
                    faces = detector.predict(frame)
    
                    if faces:
                        face = faces[0] 
                        
                        regions = {
                            'left_eye': face["left_eye"],
                            'right_eye': face["right_eye"],
                            'nose': face["nose"],
                            'mouth_left': face["left_lip"],
                            'mouth_right': face["right_lip"]
                        }
    
                        for name, point in regions.items():
                            x, y = int(point[0]), int(point[1])
                            patch_size = 40
                            x1, y1 = max(0, x - patch_size), max(0, y - patch_size)
                            x2, y2 = min(frame.shape[1], x + patch_size), min(frame.shape[0], y + patch_size)
    
                            patch_region = frame[y1:y2, x1:x2]
    
                            if patch_region.size == 0:
                                print(f"Warning: Empty patch region for {name} at frame {processed_frames + 1}")
                                continue
    
                            avg_rgb = np.mean(patch_region, axis=(0, 1)).round(0)
    
                            landmark_rgb_values[name].append(avg_rgb)
    
                    processed_frames += 1
    
                cap.release()
    
                landmarks = ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']
                rgb_array = np.array([landmark_rgb_values[landmark] for landmark in landmarks])
                rgb_array = rgb_array.transpose((0, 2, 1))
                
                rgb_array_2 = rgb_array - np.mean(rgb_array, axis = 2, keepdims = True)
                rgb_array_3 = butter_bandpass_filter(rgb_array_2, lowcut = 0.8, highcut = 6, fps = 30, order = 4)
                e1_raw = cpu_GREEN(rgb_array_3)
                e1_raw = e1_raw.reshape(5, 1, fc)
                e1 = butter_bandpass_filter(e1_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)
                
                e1_arrays = [arr.flatten() for arr in e1]
                e1_combined = np.stack(e1_arrays, axis=0)
                
                df = pd.DataFrame(e1_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e1_combined.shape[1])]
                df.columns = headers
                                
                df.to_csv(rf'D:\UBFC\P-{i}\GREEN\BVP_AVI\BVP.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e1, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr = pd.DataFrame({'BPM':bpms})
                df_hr.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr.to_csv(rf"D:\UBFC\P-{i}\GREEN\BPM_AVI\BPM.csv", index = False)
                
                rgb_array_5 = butter_bandpass_filter(rgb_array, lowcut = 0.5, highcut = 3.7, fps = 30, order = 4)
                e2_raw = cpu_PCA(rgb_array_5, component = 'all_comp')
                n_landmarks = e2_raw.size // fc 
                e2_raw = e2_raw.reshape(n_landmarks, 1, fc)
                e2 = butter_bandpass_filter(e2_raw, lowcut = 0.5, highcut = 3.7, fps = 30, order = 4)
                
                e2_arrays = [arr.flatten() for arr in e2]
                e2_combined = np.stack(e2_arrays, axis=0)
                
                df2 = pd.DataFrame(e2_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e2_combined.shape[1])]
                df2.columns = headers
                                
                df2.to_csv(rf'D:\UBFC\P-{i}\PCA\BVP_AVI\BVP.csv', index = False)

                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e2, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]

                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_2 = pd.DataFrame({'BPM':bpms})
                df_hr_2.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_2.to_csv(rf"D:\UBFC\P-{i}\PCA\BPM_AVI\BPM.csv", index = False)

                rgb_array_bp = butter_bandpass_filter(rgb_array, lowcut = 0.5, highcut = 3.5, fps = 30, order = 4)
                e3_raw = cpu_PBV(rgb_array_bp)
                e3_raw = e3_raw.reshape(5, 1, fc)
                e3 = butter_bandpass_filter(e3_raw, lowcut = 0.5, highcut = 3.5, fps = 30, order = 6)

                e3_arrays = [arr.flatten() for arr in e3]
                e3_combined = np.stack(e3_arrays, axis = 0)
                
                df3 = pd.DataFrame(e3_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e3_combined.shape[1])]
                df3.columns = headers
                                
                df3.to_csv(rf'D:\UBFC\P-{i}\PBV\BVP_AVI\BVP.csv', index = False)
                
                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e3, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_3 = pd.DataFrame({'BPM':bpms})
                df_hr_3.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_3.to_csv(rf"D:\UBFC\P-{i}\PBV\BPM_AVI\BPM.csv", index = False)
            
                rgb_array_6 = rgb_array/np.mean(rgb_array, axis = 2, keepdims = True)
                rgb_array_7 = butter_bandpass_filter_PCA(rgb_array_6, lowcut = 0.5, highcut = 3.7, fps = 30, order = 4)
                e4_raw = cpu_CHROM(rgb_array_6)
                e4_raw = e4_raw.reshape(5, 1, fc)
                e4 = butter_bandpass_filter(e4_raw, lowcut = 0.67, highcut = 4.00, fps = 30, order = 4)

                e4_arrays = [arr.flatten() for arr in e4]
                e4_combined = np.stack(e4_arrays, axis=0)
                
                df4 = pd.DataFrame(e4_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e4_combined.shape[1])]
                df4.columns = headers
                                
                df4.to_csv(rf'D:\UBFC\P-{i}\CHROM\BVP_AVI\BVP.csv', index = False)
                
                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e4, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_4 = pd.DataFrame({'BPM':bpms})
                df_hr_4.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_4.to_csv(rf"D:\UBFC\P-{i}\CHROM\BPM_AVI\BPM.csv", index = False)
           
                e5_raw = cpu_POS(rgb_array)
                e5_raw = e5_raw.reshape(5, 1, fc)
                e5 = butter_bandpass_filter(e5_raw, lowcut = 0.75, highcut = 3.33, fps = 30, order = 4)

                e5_arrays = [arr.flatten() for arr in e5]
                e5_combined = np.stack(e5_arrays, axis=0)
            
                df5 = pd.DataFrame(e5_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e5_combined.shape[1])]
                df5.columns = headers
                                
                df5.to_csv(rf'D:\UBFC\P-{i}\POS\BVP_AVI\BVP.csv', index = False)
                
                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e5, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_5 = pd.DataFrame({'BPM':bpms})
                df_hr_5.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_5.to_csv(rf"D:\UBFC\P-{i}\POS\BPM_AVI\BPM.csv", index = False)
            
                e6_raw = cpu_LGI(rgb_array)
                e6_raw = e6_raw.reshape(5, 1, fc)
                e6 = butter_bandpass_filter(e5_raw, lowcut = 0.5, highcut = 2.5, fps = 30, order = 4)

                e6_arrays = [arr.flatten() for arr in e6]
                e6_combined = np.stack(e6_arrays, axis=0)
            
                df6 = pd.DataFrame(e6_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e6_combined.shape[1])]
                df6.columns = headers
                                
                df6.to_csv(rf'D:\UBFC\P-{i}\LGI\BVP_AVI\BVP.csv', index = False)
                
                fps = 30
                nperseg = 256
                noverlap = int(nperseg * 0.9)
                window = hann(int(nperseg))
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e6, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_6 = pd.DataFrame({'BPM':bpms})
                df_hr_6.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_6.to_csv(rf"D:\UBFC\P-{i}\LGI\BPM_AVI\BPM.csv", index = False)
                
                rgb_array_9 = butter_bandpass_filter_OMIT(rgb_array, lowcut = 0.75, highcut = 4, fps = 30, order = 4, alpha = 25)
                e7_raw = cpu_OMIT(rgb_array_9)
                e7_raw = e7_raw.reshape(5, 1, fc)
                e7 = butter_bandpass_filter(e7_raw, lowcut = 0.75, highcut = 4.00, fps = 30, order = 4)

                e7_arrays = [arr.flatten() for arr in e7]
                e7_combined = np.stack(e7_arrays, axis=0)
            
                df7 = pd.DataFrame(e7_combined)
                
                headers = [f"Frame_{j+1}" for j in range(e7_combined.shape[1])]
                df7.columns = headers
                                
                df7.to_csv(rf'D:\UBFC\P-{i}\OMIT\BVP_AVI\BVP.csv', index = False)
                
                fps = 30
                nperseg = 300
                noverlap = int(nperseg * 0.9)
                window = kaiser(int(nperseg), beta = 25)
                minHz = 0.75
                maxHz = 3.33
                F, P = welch(e7, fs = fps, nperseg = nperseg, noverlap = noverlap, window = window,  nfft = 512)
                F = F.astype(np.float32)
                P = P.astype(np.float32)
                band = np.argwhere((F > minHz) & (F < maxHz)).flatten()
                Pfreqs = 60 * F[band]
                Power = P[:, 0, band]
                Pmax = np.argmax(Power, axis = 1)
                bpms = (np.round(Pfreqs[Pmax.squeeze()]))
                
                df_hr_7 = pd.DataFrame({'BPM':bpms})
                df_hr_7.at[0, 'Mean BPM'] = np.round(np.mean(bpms))
                
                df_hr_7.to_csv(rf"D:\UBFC\P-{i}\OMIT\BPM_AVI\BPM.csv", index = False)
            
        except (ValueError, OSError) as e:
                print(e)
                pass

**pyVHR Pipeline**

In [ ]:
# %%time
import mediapipe
import torch
import csv
import pandas as pd
import os
import numpy as np
from inspect import getmembers, isfunction
from importlib import import_module
import pyVHR as vhr
from pyVHR.extraction.sig_processing import *
from pyVHR.extraction.sig_extraction_methods import *
from pyVHR.extraction.skin_extraction_methods import *
from pyVHR.BVP.BVP import *
from pyVHR.BPM.BPM import *
from pyVHR.BVP.methods import *
from pyVHR.BVP.filters import *
from pyVHR.deepRPPG.mtts_can import *
from pyVHR.deepRPPG.hr_cnn import *
from pyVHR.extraction.utils import *

class Pipeline_3():
    """ 
    This class runs the pyVHR pipeline on a single video or dataset
    """

    minHz = 0.75
    maxHz = 4.0

    def __init__(self):
        pass

    def run_on_video_multimethods(self, videoFileName, 
                                  winsize = 8, 
                                  ldmks_list=None,
                                  cuda=True, 
                                  roi_method='convexhull', 
                                  roi_approach='holistic', 
                                  methods=['cpu_CHROM', 'cpu_POS', 'cpu_LGI'], 
                                  estimate='holistic', 
                                  movement_thrs=[10, 5, 2],
                                  patch_size=28, 
                                  RGB_LOW_HIGH_TH = (5, 240),
                                  Skin_LOW_HIGH_TH = (5, 240),
                                  pre_filt=False, 
                                  post_filt=True, 
                                  verb=True):
        """ 
        Modified to return BPM values separately for each method.
        """
        # set landmark list
        if not ldmks_list:
            ldmks_list = [2, 3, 4, 5, 6, 8, 9, 10, 18, 21, 32, 35, 36, 43, 46, 47, 48, 50, 54, 58, 67, 68, 69, 71, 92, 93, 101, 103, 104, 108, 109, 116, 117, 118, 123, 132, 134, 135, 138, 139, 142, 148, 149, 150, 151, 152, 182, 187, 188, 193, 197, 201, 205, 206, 207, 210, 211, 212, 216, 234, 248, 251, 262, 265, 266, 273, 277, 278, 280, 284, 288, 297, 299, 322, 323, 330, 332, 333, 337, 338, 345, 346, 361, 363, 364, 367, 368, 371, 377, 379, 411, 412, 417, 421, 425, 426, 427, 430, 432, 436]
        
        # test video filename
        assert os.path.isfile(videoFileName), "The video file does not exist!"
        
        sig_processing = SignalProcessing()
        av_meths = getmembers(import_module('pyVHR.BVP.methods'), isfunction)
        available_methods = [am[0] for am in av_meths]

        for m in methods:
            assert m in available_methods, "\nrPPG method not recognized!!"

        if cuda:
            sig_processing.display_cuda_device()
            sig_processing.choose_cuda_device(0)
        
        ## 1. set skin extractor
        target_device = 'GPU' if cuda else 'CPU'
        if roi_method == 'convexhull':
            sig_processing.set_skin_extractor(SkinExtractionConvexHull(target_device))
        elif roi_method == 'faceparsing':
            sig_processing.set_skin_extractor(SkinExtractionFaceParsing(target_device))
        else:
            raise ValueError("Unknown 'roi_method'")
              
        ## 2. set patches
        if roi_approach == 'patches':
            sig_processing.set_landmarks(ldmks_list)
            sig_processing.set_square_patches_side(float(patch_size))
        
        # set sig-processing and skin-processing params
        SignalProcessingParams.RGB_LOW_TH = RGB_LOW_HIGH_TH[0]
        SignalProcessingParams.RGB_HIGH_TH = RGB_LOW_HIGH_TH[1]
        SkinProcessingParams.RGB_LOW_TH = Skin_LOW_HIGH_TH[0]
        SkinProcessingParams.RGB_HIGH_TH = Skin_LOW_HIGH_TH[1]

        if verb:
            print('\nProcessing Video ' + videoFileName)
        fps = get_fps(videoFileName)
        sig_processing.set_total_frames(0)

        ## 3. ROI selection
        if verb:
            print('\nROI processing...')
        sig = []
        if roi_approach == 'holistic':
            # SIG extraction with holistic
            sig = sig_processing.extract_holistic(videoFileName)
        elif roi_approach == 'patches':
            # SIG extraction with patches
            sig = sig_processing.extract_patches(videoFileName, 'squares', 'mean')
        if verb:
            print(' - Extraction approach: ' + roi_approach)

        ## 4. Signal windowing
        windowed_sig, timesES = sig_windowing(sig, winsize, 1, fps)
        if verb:
            print(f' - Number of windows: {len(windowed_sig)}')
            print(' - Win size: (#ROI, #landmarks, #frames) = ', windowed_sig[0].shape)


        ## 5. PRE FILTERING
        if verb:
            print('\nPre filtering...')
        filtered_windowed_sig = windowed_sig

        if pre_filt:
            module = import_module('pyVHR.BVP.filters')
            method_to_call = getattr(module, 'BPfilter')
            filtered_windowed_sig = apply_filter(filtered_windowed_sig,
                                                    method_to_call, 
                                                    fps=fps, 
                                                    params={'minHz':Pipeline_3.minHz, 
                                                            'maxHz':Pipeline_3.maxHz, 
                                                            'fps':'adaptive', 
                                                            'order':6})
            if verb:
                print(f' - Pre-filter applied: {method_to_call.__name__}')

        ## 6. BVP extraction for multiple methods
        bvps_win = []
        bpmES_per_method = {}  # To store BPMs for each method
        
        for method in methods:
            if verb:
                print("\nBVP extraction...")
                print(" - Extraction method: " + method)
            module = import_module('pyVHR.BVP.methods')
            method_to_call = getattr(module, method)
            
            if 'cpu' in method:
                method_device = 'cpu'
            elif 'torch' in method:
                method_device = 'torch'
            elif 'cupy' in method:
                method_device = 'cuda'

            if 'POS' in method:
                pars = {'fps':'adaptive'}
            elif 'PCA' in method or 'ICA' in method:
                pars = {'component': 'second_comp'}
            else:
                pars = {}

            bvps_win_m = RGB_sig_to_BVP(filtered_windowed_sig, 
                                        fps, device_type=method_device, 
                                        method=method_to_call, params=pars)
            print(bvps_win_m[0].shape)

            ## 7. POST FILTERING
            if post_filt:
                module = import_module('pyVHR.BVP.filters')
                method_to_call = getattr(module, 'BPfilter')
                bvps_win_m = apply_filter(bvps_win_m, 
                                        method_to_call, 
                                        fps=fps, 
                                        params={'minHz':Pipeline_3.minHz, 
                                                'maxHz':Pipeline_3.maxHz, 
                                                'fps':'adaptive', 
                                                'order':6})
            if verb:
                print(f' - Post-filter applied: {method_to_call.__name__}')

            # Store the BVP signals for each method
            if len(bvps_win) == 0:
                bvps_win = bvps_win_m
            else:
                for i in range(len(bvps_win_m)):
                    bvps_win[i] = np.concatenate((bvps_win[i], bvps_win_m[i]))

            ## 8. BPM extraction for each method
            if roi_approach == 'holistic':
                if cuda:
                    bpmES = BVP_to_BPM_cuda(bvps_win_m, fps, minHz=Pipeline_3.minHz, maxHz=Pipeline_3.maxHz)
                else:
                    bpmES = BVP_to_BPM(bvps_win_m, fps, minHz=Pipeline_3.minHz, maxHz=Pipeline_3.maxHz)

            elif roi_approach == 'patches':
                if estimate == 'clustering':
                    ma = MotionAnalysis(sig_processing, winsize, fps)
                    bpmES = BPM_clustering(ma, bvps_win_m, fps, winsize, movement_thrs=movement_thrs, opt_factor=0.5)
                elif estimate == 'median':
                    bpmES = BVP_to_BPM(bvps_win_m, fps, minHz=Pipeline_3.minHz, maxHz=Pipeline_3.maxHz)
                    bpmES, _ = BPM_median(bpmES)

            bpmES_per_method[method] = bpmES

        return bvps_win, timesES, bpmES_per_method

pipeline = Pipeline_3()

for l in range(1, 9):
    for k in range(1, 22):
        try:
            video_file = rf'D:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-{l}\Lossless\Window_{k}.avi'
            window_size = 3
            # methods = ['cpu_PCA']
            methods = ['cpu_PBV', 'cpu_OMIT']
            
            bvps_win, timesES, bpmES_per_method = pipeline.run_on_video_multimethods(
                videoFileName=video_file,
                winsize=window_size,
                methods=methods,
                cuda=False,
                roi_method='convexhull',
                roi_approach='holistic',
                estimate='holistic',
                verb=True
            )
            
            for method in bpmES_per_method:
                bpm_list = bpmES_per_method[method]
                
                rounded_bpm_list = [np.round(bpm) for bpm in bpm_list]
                
                rounded_mean_bpm = np.round(np.mean(bpm_list))
                
                csv_file_name = rf'D:\HRV_New\Data\Dynamic + Static\Algorithms\{method.replace("cpu_", "")}\P-{l}\BPM_20_AVI\BPM_pyVHR_Window_{k}.csv'
                
                df = pd.DataFrame({
                    'Window': range(1, len(rounded_bpm_list) + 1),
                    'Rounded BPM': rounded_bpm_list,
                    'Mean BPM': [''] * len(rounded_bpm_list)
                })
                
                df.at[0, 'Mean BPM'] = rounded_mean_bpm
                print(method, rounded_mean_bpm)
                df.to_csv(csv_file_name, index = False)
        except (ValueError, OSError, AssertionError) as e:
            print(e)
            pass


Processing Video D:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-1\Lossless\Window_1.avi

ROI processing...
 - Extraction approach: holistic
 - Number of windows: 18
 - Win size: (#ROI, #landmarks, #frames) =  (1, 3, 90)

Pre filtering...

BVP extraction...
 - Extraction method: cpu_PBV
(1, 90)
 - Post-filter applied: BPfilter

BVP extraction...
 - Extraction method: cpu_OMIT
(1, 90)
 - Post-filter applied: BPfilter
cpu_PBV 62.0
cpu_OMIT 65.0

Processing Video D:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-1\Lossless\Window_2.avi

ROI processing...
 - Extraction approach: holistic
 - Number of windows: 18
 - Win size: (#ROI, #landmarks, #frames) =  (1, 3, 90)

Pre filtering...

BVP extraction...
 - Extraction method: cpu_PBV
(1, 90)
 - Post-filter applied: BPfilter

BVP extraction...
 - Extraction method: cpu_OMIT
(1, 90)
 - Post-filter applied: BPfilter
cpu_PBV 61.0
cpu_OMIT 61.0

Processing Video D:\HRV_New\Video Database\Dynamic + Static\Face Videos\480

D:\Anaconda\envs\pyvhr\lib\site-packages\pyVHR\BVP\methods.py:203: RuntimeWarning: invalid value encountered in divide
  signal_norm_r = signal[:,0,:] / np.expand_dims(sig_mean[:,0],axis=1)
D:\Anaconda\envs\pyvhr\lib\site-packages\pyVHR\BVP\methods.py:204: RuntimeWarning: invalid value encountered in divide
  signal_norm_g = signal[:,1,:] / np.expand_dims(sig_mean[:,1],axis=1)
D:\Anaconda\envs\pyvhr\lib\site-packages\pyVHR\BVP\methods.py:205: RuntimeWarning: invalid value encountered in divide
  signal_norm_b = signal[:,2,:] / np.expand_dims(sig_mean[:,2],axis=1)


 - Extraction approach: holistic
 - Number of windows: 18
 - Win size: (#ROI, #landmarks, #frames) =  (1, 3, 90)

Pre filtering...

BVP extraction...
 - Extraction method: cpu_PBV
(0, 1)
The length of the input vector x must be greater than padlen, which is 39.
The video file does not exist!
The video file does not exist!
The video file does not exist!

Processing Video D:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-2\Lossless\Window_1.avi

ROI processing...
 - Extraction approach: holistic
 - Number of windows: 18
 - Win size: (#ROI, #landmarks, #frames) =  (1, 3, 90)

Pre filtering...

BVP extraction...
 - Extraction method: cpu_PBV
(1, 90)
 - Post-filter applied: BPfilter

BVP extraction...
 - Extraction method: cpu_OMIT
(1, 90)
 - Post-filter applied: BPfilter
cpu_PBV 92.0
cpu_OMIT 87.0

Processing Video D:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-2\Lossless\Window_2.avi

ROI processing...
 - Extraction approach: holistic
 - Number of windows: 18
 -

In [9]:
import subprocess
import os

# Define the base directory and file naming pattern
for x in range(3, 9):
    base_dir = rf"F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-{x}\P-{x}_Split_20"
    output_dir = os.path.join(base_dir, "Converted_AVI")
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Loop through all files with the naming pattern
    for l in range(1, 22):
        input_file = os.path.join(base_dir, f"Window_{l}.mp4")
        output_file = os.path.join(output_dir, f"Window_{l}.avi")
        
        # FFmpeg command for conversion with yuv420p
        command = [
            "ffmpeg",
            "-i", input_file,
            "-c:v", "rawvideo",  # Raw uncompressed video
            "-pix_fmt", "yuv420p",  # Specify YUV 4:2:0 pixel format
            "-an",  # Remove audio (optional, remove this if audio is needed)
            output_file
        ]
        
        try:
            # Run the FFmpeg command
            print(f"Converting: {input_file} -> {output_file}")
            subprocess.run(command, check=True)
            print(f"Successfully converted {input_file} to {output_file}")
        except subprocess.CalledProcessError as e:
            print(f"Error converting {input_file}: {e}")
    
    print("Batch conversion completed!")

Converting: F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Window_1.mp4 -> F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Converted_AVI\Window_1.avi
Successfully converted F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Window_1.mp4 to F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Converted_AVI\Window_1.avi
Converting: F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Window_2.mp4 -> F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Converted_AVI\Window_2.avi
Successfully converted F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Window_2.mp4 to F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Converted_AVI\Window_2.avi
Converting: F:\HRV_New\Video Database\Dynamic + Static\Face Videos\480P\P-3\P-3_Split_20\Window_3.mp4 -> F:\HRV_New\Video Database\Dynamic + S

In [44]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
     nyq = 0.5 * fs
     low = lowcut / nyq
     high = highcut / nyq
     b, a = butter(order, [low, high], btype='band')
     return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

bpms = []
videos = []
for k in range(29, 43):
    for l in range(1, 13):
        bvp_csv_path = rf'D:\UBFC\Testing\P-{k}\LSTC-rPPG\BVP_AVI\BVP_Trained_Clip_2_{l}.csv'
        df = pd.read_csv(bvp_csv_path) 
        bvp_signal = df['Pulse'].values  

        lowcut = 0.5
        highcut = 5.0
        fps = 29
        order = 6
        winsize = 5

        filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
        
        max_freqs = []
        window_size = winsize * fps 
        
        for start in range(0, len(filtered_bvp) - window_size + 1, 24):
            windowed_signal = filtered_bvp[start:start + window_size]
            f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size)
            max_freq = f[np.argmax(Pxx)]
            max_freqs.append(max_freq * 60)
        
        mean_freq = np.mean(max_freqs)
        bpms.append(round(mean_freq))
        videos.append(k)
        print(f'{k} = {mean_freq:.0f}')
        
df = pd.DataFrame({'Video':videos ,'BPM':bpms})
df.to_csv(r'D:\UBFC\Testing\Predictions_2.csv', index = False)

29 = 120
29 = 120
29 = 120
29 = 120
29 = 120
29 = 120
29 = 120
29 = 120
29 = 120
29 = 120
29 = 120
29 = 120
30 = 120
30 = 120
30 = 120
30 = 120
30 = 120
30 = 120
30 = 120
30 = 120
30 = 120
30 = 120
30 = 120
30 = 120
31 = 120
31 = 120
31 = 120
31 = 120
31 = 120
31 = 120
31 = 120
31 = 120
31 = 120
31 = 120
31 = 120
31 = 120
32 = 120
32 = 120
32 = 120
32 = 120
32 = 120
32 = 120
32 = 120
32 = 120
32 = 120
32 = 120
32 = 120
32 = 120
33 = 120
33 = 120
33 = 120
33 = 120
33 = 120
33 = 120
33 = 120
33 = 120
33 = 120
33 = 120
33 = 120
33 = 120
34 = 120
34 = 120
34 = 120
34 = 120
34 = 120
34 = 120
34 = 120
34 = 120
34 = 120
34 = 120
34 = 120
34 = 120
35 = 120
35 = 120
35 = 120
35 = 120
35 = 120
35 = 120
35 = 120
35 = 120
35 = 120
35 = 120
35 = 120
35 = 120
36 = 120
36 = 120
36 = 120
36 = 120
36 = 120
36 = 120
36 = 120
36 = 120
36 = 120
36 = 120
36 = 120
36 = 120
37 = 120
37 = 120
37 = 120
37 = 120
37 = 120
37 = 120
37 = 120
37 = 120
37 = 120
37 = 120
37 = 120
37 = 120
38 = 120
38 = 120
38 = 120
3

In [3]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch


def butter_bandpass(lowcut, highcut, fs, order=6):
     nyq = 0.5 * fs
     low = lowcut / nyq
     high = highcut / nyq
     b, a = butter(order, [low, high], btype='band')
     return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y
    
# Load the CSV file
for k in range(20, 23):
    if(k!=3 and k!=4 and k!=5 and k!=6):
        bvp_csv_path = rf'D:\UBFC\P-{k}\PhysNet\BVP_AVI\BVP_Frame1_LowestTrainLoss.csv'
        df = pd.read_csv(bvp_csv_path)  # Load the CSV file
        bvp_signal = df['Pulse'].values  # Assuming the BVP signal is in a column named 'BVP'

# Butterworth bandpass filter parameters
        lowcut = 0.7
        highcut = 4.0
        fps = 23
        order = 6
        winsize = 8
        
        # Design Butterworth bandpass filter

        
        # Apply the bandpass filter to the BVP signal
        filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
        
        # Split the filtered signal into windows and compute PSD
        max_freqs = []
        window_size = winsize * fps  # Convert window size to samples
        
        # Create windows where each window overlaps by 1 second
        for start in range(0, len(filtered_bvp) - window_size + 1, 24):
            windowed_signal = filtered_bvp[start:start + window_size]
            # Compute the Power Spectral Density using Welch's method
            f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size)
            # Find the frequency corresponding to the maximum power
            max_freq = f[np.argmax(Pxx)]
            max_freqs.append(max_freq * 60)  # Multiply by 60 to convert to BPM
        
        # Calculate the mean of maximum frequencies
        mean_freq = np.mean(max_freqs)
        
        # Print the results for each window and the mean frequency
        # for idx, freq in enumerate(max_freqs):
        #     print(f'Window {idx + 1}: Max frequency (BPM) = {freq:.2f}')
        print(f'{k} = {mean_freq:.0f}')

20 = 89
21 = 80
22 = 85


In [20]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
     nyq = 0.5 * fs
     low = lowcut / nyq
     high = highcut / nyq
     b, a = butter(order, [low, high], btype='band')
     return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(29, 43):
    for p in range(1, 17):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\Split_Testing_Phys\P-{k}\clip_{p}_BVP.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['BVP'].values
    
            lowcut = 0.7
            highcut = 4.0
            fps = 29
            order = 6
            winsize = 4
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
            
            max_freqs = []
            window_size = winsize * fps
            
            for start in range(0, len(filtered_bvp) - window_size + 1, 24):
                windowed_signal = filtered_bvp[start:start + window_size]
                f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size, nfft = 2048)
                max_freq = f[np.argmax(Pxx)]
                max_freqs.append(max_freq * 60)
            
            mean_freq = np.mean(max_freqs)
            bpms.append(round(mean_freq))
            videos.append(k)
    
            print(f'{k} = {mean_freq:.0f}')
        except (ValueError, OSError) as e:
            pass
    
df = pd.DataFrame({'Video':videos ,'BPM':bpms})
df.to_csv(r'D:\UBFC\Testing\GT_4.csv', index = False)

29 = 107
29 = 94
29 = 110
29 = 111
29 = 118
29 = 116
29 = 122
29 = 119
29 = 118
29 = 122
29 = 116
29 = 116
29 = 118
29 = 116
29 = 121
29 = 123
30 = 60
30 = 69
30 = 59
30 = 63
30 = 56
30 = 59
30 = 55
30 = 51
30 = 52
30 = 53
30 = 53
30 = 55
30 = 53
30 = 51
30 = 56
31 = 108
31 = 108
31 = 107
31 = 110
31 = 107
31 = 108
31 = 107
31 = 109
31 = 106
31 = 99
31 = 103
31 = 104
31 = 106
31 = 105
31 = 109
31 = 102
32 = 81
32 = 79
32 = 80
32 = 88
32 = 83
32 = 82
32 = 79
32 = 85
32 = 74
32 = 82
32 = 81
32 = 83
32 = 80
32 = 76
32 = 85
33 = 83
33 = 82
33 = 84
33 = 88
33 = 83
33 = 85
33 = 86
33 = 81
33 = 82
33 = 86
33 = 94
33 = 83
33 = 82
33 = 82
33 = 77
34 = 80
34 = 102
34 = 100
34 = 94
34 = 92
34 = 100
34 = 121
34 = 104
34 = 102
34 = 93
34 = 97
34 = 91
34 = 97
34 = 99
34 = 99
35 = 79
35 = 77
35 = 88
35 = 86
35 = 95
35 = 90
35 = 97
35 = 93
35 = 94
35 = 97
35 = 93
35 = 96
35 = 98
35 = 93
35 = 93
36 = 97
36 = 90
36 = 91
36 = 99
36 = 100
36 = 91
36 = 89
36 = 93
36 = 93
36 = 92
36 = 90
36 = 97
36 = 99
36 

In [19]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
     nyq = 0.5 * fs
     low = lowcut / nyq
     high = highcut / nyq
     b, a = butter(order, [low, high], btype='band')
     return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(29, 43):
    for p in range(1, 17):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\P-{k}\PhysNet\BVP_AVI\BVP_128Frames_{p}.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['Pulse'].values
    
            lowcut = 0.7
            highcut = 4.0
            fps = 29
            order = 6
            winsize = 4
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
            
            max_freqs = []
            window_size = winsize * fps
            
            for start in range(0, len(filtered_bvp) - window_size + 1, 24):
                windowed_signal = filtered_bvp[start:start + window_size]
                f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size, nfft = 2048)
                max_freq = f[np.argmax(Pxx)]
                max_freqs.append(max_freq * 60)
            
            mean_freq = np.mean(max_freqs)
            bpms.append(round(mean_freq))
            videos.append(k)
    
            print(f'{k} = {mean_freq:.0f}')
        except (ValueError, OSError) as e:
            pass
    
df = pd.DataFrame({'Video':videos ,'BPM':bpms})
df.to_csv(r'D:\UBFC\Testing\Preds_4.csv', index = False)

29 = 71
29 = 111
29 = 68
29 = 118
29 = 119
29 = 88
29 = 83
29 = 107
29 = 113
29 = 108
29 = 105
29 = 111
29 = 110
29 = 102
29 = 100
29 = 116
30 = 116
30 = 74
30 = 82
30 = 109
30 = 92
30 = 82
30 = 105
30 = 79
30 = 97
30 = 63
30 = 83
30 = 104
30 = 71
30 = 86
30 = 82
31 = 122
31 = 96
31 = 86
31 = 91
31 = 104
31 = 88
31 = 119
31 = 95
31 = 110
31 = 91
31 = 114
31 = 88
31 = 66
31 = 134
31 = 95
31 = 88
32 = 120
32 = 96
32 = 145
32 = 82
32 = 148
32 = 81
32 = 140
32 = 88
32 = 71
32 = 93
32 = 120
32 = 107
32 = 82
32 = 149
32 = 106
33 = 84
33 = 138
33 = 123
33 = 83
33 = 126
33 = 119
33 = 107
33 = 105
33 = 116
33 = 42
33 = 79
33 = 104
33 = 94
33 = 132
33 = 109
34 = 64
34 = 95
34 = 78
34 = 64
34 = 97
34 = 92
34 = 91
34 = 101
34 = 70
34 = 70
34 = 65
34 = 86
34 = 80
34 = 110
34 = 143
35 = 81
35 = 104
35 = 111
35 = 110
35 = 114
35 = 86
35 = 109
35 = 88
35 = 92
35 = 66
35 = 98
35 = 99
35 = 110
35 = 118
35 = 105
36 = 85
36 = 62
36 = 82
36 = 88
36 = 99
36 = 99
36 = 82
36 = 65
36 = 104
36 = 77
36 = 107
36 

In [23]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(1, 29): 
    for p in range(1, 20):
        try:
            bvp_csv_path = rf'D:\UBFC\Training\Split_Training_2\P-{k}\clip_{p}_BVP.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['BVP'].values
    
            lowcut = 0.7
            highcut = 4.0  
            fps = 29 
            order = 6  
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
    
            window_size = int(160)  
            if len(filtered_bvp) < window_size:
                print(f"Skipping P-{k} Clip-{p}: Not enough data for the window size.")
                continue
            
            windowed_signal = filtered_bvp[:window_size]
            
            f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size)
            max_freq = f[np.argmax(Pxx)]  
            bpm = max_freq * 60  
            
            bpms.append(round(bpm))
            videos.append(k)
    
            print(f'P-{k} Clip-{p} = {bpm:.0f} BPM')
            
        except (ValueError, OSError) as e:
            print(e)
            pass

df = pd.DataFrame({'Video': videos, 'BPM': bpms})
df.to_csv(r'D:\UBFC\Testing\XTrain.csv', index=False)

P-1 Clip-1 = 98 BPM
P-1 Clip-2 = 98 BPM
P-1 Clip-3 = 98 BPM
P-1 Clip-4 = 109 BPM
P-1 Clip-5 = 109 BPM
P-1 Clip-6 = 109 BPM
P-1 Clip-7 = 109 BPM
P-1 Clip-8 = 120 BPM
P-1 Clip-9 = 109 BPM
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\Split_Training_2\\P-1\\clip_10_BVP.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\Split_Training_2\\P-1\\clip_11_BVP.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\Split_Training_2\\P-1\\clip_12_BVP.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\Split_Training_2\\P-1\\clip_13_BVP.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\Split_Training_2\\P-1\\clip_14_BVP.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\Split_Training_2\\P-1\\clip_15_BVP.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\Split_Training_2\\P-1\\clip_16_BVP.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\Split_Training_2\\P-1\\clip_17_BVP.csv'
[Errno 2] No such file or dire

In [26]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(1, 29): 
    for p in range(1, 13):
        try:
            bvp_csv_path = rf'D:\UBFC\Training\P-{k}\LSTC-rPPG\BVP_AVI\BVP_Trained_Clip_{p}.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['Pulse'].values
    
            lowcut = 0.7
            highcut = 4.0  
            fps = 29 
            order = 6  
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
    
            window_size = int(160)  
            if len(filtered_bvp) < window_size:
                print(f"Skipping P-{k} Clip-{p}: Not enough data for the window size.")
                continue
            
            windowed_signal = filtered_bvp[:window_size]
            
            f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size)
            max_freq = f[np.argmax(Pxx)]  
            bpm = max_freq * 60  
            
            bpms.append(round(bpm))
            videos.append(k)
    
            print(f'P-{k} Clip-{p} = {bpm:.0f} BPM')

        except (ValueError, OSError) as e:
            print(e)
            pass

df = pd.DataFrame({'Video': videos, 'BPM': bpms})
df.to_csv(r'D:\UBFC\Testing\XTrain_Model.csv', index=False)

P-1 Clip-1 = 120 BPM
P-1 Clip-2 = 120 BPM
P-1 Clip-3 = 120 BPM
P-1 Clip-4 = 120 BPM
P-1 Clip-5 = 120 BPM
P-1 Clip-6 = 120 BPM
P-1 Clip-7 = 120 BPM
P-1 Clip-8 = 120 BPM
P-1 Clip-9 = 120 BPM
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\P-1\\LSTC-rPPG\\BVP_AVI\\BVP_Trained_Clip_10.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\P-1\\LSTC-rPPG\\BVP_AVI\\BVP_Trained_Clip_11.csv'
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\P-1\\LSTC-rPPG\\BVP_AVI\\BVP_Trained_Clip_12.csv'
P-2 Clip-1 = 120 BPM
P-2 Clip-2 = 120 BPM
P-2 Clip-3 = 120 BPM
P-2 Clip-4 = 120 BPM
P-2 Clip-5 = 120 BPM
P-2 Clip-6 = 120 BPM
P-2 Clip-7 = 120 BPM
P-2 Clip-8 = 120 BPM
P-2 Clip-9 = 120 BPM
P-2 Clip-10 = 120 BPM
P-2 Clip-11 = 120 BPM
[Errno 2] No such file or directory: 'D:\\UBFC\\Training\\P-2\\LSTC-rPPG\\BVP_AVI\\BVP_Trained_Clip_12.csv'
P-3 Clip-1 = 120 BPM
P-3 Clip-2 = 120 BPM
P-3 Clip-3 = 120 BPM
P-3 Clip-4 = 120 BPM
P-3 Clip-5 = 120 BPM
P-3 Clip-6 = 120 BPM
P-3 Clip-7 = 120 BPM

In [12]:
import numpy as np
import pandas as pd
import os

def get_HR(sig, fps):
    """
    Compute Heart Rate (HR) from a signal using FFT.
    
    Args:
        sig (np.ndarray): Input signal (1D array).
        fps (float): Frames per second (sampling frequency).
    
    Returns:
        int: Estimated heart rate (HR) in beats per minute (BPM).
    """
    
    sig = (sig - np.mean(sig)) / (np.std(sig) + 1e-6)  # Mean subtraction and standardization

    sig = np.squeeze(sig)
    if sig.ndim != 1:
        raise ValueError("Input signal must be a 1D array.")

    N = len(sig)  # Use the length of the signal directly
    m2 = 2 ** int(np.ceil(np.log2(N)))  # Next power of 2 for zero-padding
    
    amp = np.fft.fft(sig, n=m2)
    pows = np.abs(amp[:m2 // 2 + 1]) ** 2  # Use only the positive frequencies
    
    freqs = fps * np.arange(0, m2 // 2 + 1) / m2
    
    valid_mask = (freqs >= 0.7) & (freqs <= 4.0)
    pows = pows[valid_mask]
    freqs = freqs[valid_mask]
    
    if len(freqs) == 0 or len(pows) == 0:
        raise ValueError("No valid frequencies found in the desired range.")
    
    idx = np.argmax(pows)
    HR_freq = freqs[idx]
    
    HR = int(np.ceil(HR_freq * 60))
    
    return HR

fps = 30 

results = [] 

for k in range(36, 43): 
    for l in range(1, 69):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\Split_Testing_JAMS\P-{k}\clip_{l}_BVP.csv'
            
            if not os.path.exists(bvp_csv_path):
                print(f"File not found: {bvp_csv_path}")
                continue
            
            df = pd.read_csv(bvp_csv_path)
            
            if 'BVP' not in df.columns:
                print(f"Column 'Pulse' missing in file: {bvp_csv_path}")
                continue
            
            bvp_signal = df['BVP'].values
            
            HR = get_HR(bvp_signal, fps)
            
            # Store results
            results.append({
                "Participant": k,
                "Clip": l,
                "HR": HR
            })
            print(f"Processed: Participant-{k}, Clip-{l}, HR={HR} BPM")
        
        except Exception as e:
            print(f"Error processing file {bvp_csv_path}: {e}")

results_df = pd.DataFrame(results)

results_df.to_csv(r'D:\UBFC\Testing\GT.csv', index=False)

Processed: Participant-36, Clip-1, HR=99 BPM
Processed: Participant-36, Clip-2, HR=99 BPM
Processed: Participant-36, Clip-3, HR=99 BPM
Processed: Participant-36, Clip-4, HR=92 BPM
Processed: Participant-36, Clip-5, HR=92 BPM
Processed: Participant-36, Clip-6, HR=92 BPM
Processed: Participant-36, Clip-7, HR=92 BPM
Processed: Participant-36, Clip-8, HR=92 BPM
Processed: Participant-36, Clip-9, HR=92 BPM
Processed: Participant-36, Clip-10, HR=99 BPM
Processed: Participant-36, Clip-11, HR=106 BPM
Processed: Participant-36, Clip-12, HR=106 BPM
Processed: Participant-36, Clip-13, HR=106 BPM
Processed: Participant-36, Clip-14, HR=99 BPM
Processed: Participant-36, Clip-15, HR=99 BPM
Processed: Participant-36, Clip-16, HR=99 BPM
Processed: Participant-36, Clip-17, HR=106 BPM
Processed: Participant-36, Clip-18, HR=106 BPM
Processed: Participant-36, Clip-19, HR=106 BPM
Processed: Participant-36, Clip-20, HR=99 BPM
Processed: Participant-36, Clip-21, HR=99 BPM
Processed: Participant-36, Clip-22, H

In [18]:
import numpy as np
import pandas as pd
import os

def get_HR(sig, fps):
    """
    Compute Heart Rate (HR) from a signal using FFT.
    
    Args:
        sig (np.ndarray): Input signal (1D array).
        fps (float): Frames per second (sampling frequency).
    
    Returns:
        int: Estimated heart rate (HR) in beats per minute (BPM).
    """
    
    sig = (sig - np.mean(sig)) / (np.std(sig) + 1e-6)  # Mean subtraction and standardization

    sig = np.squeeze(sig)
    if sig.ndim != 1:
        raise ValueError("Input signal must be a 1D array.")

    N = len(sig)  # Use the length of the signal directly
    m2 = 2 ** int(np.ceil(np.log2(N)))  # Next power of 2 for zero-padding
    
    amp = np.fft.fft(sig, n=m2)
    pows = np.abs(amp[:m2 // 2 + 1]) ** 2  # Use only the positive frequencies
    
    freqs = fps * np.arange(0, m2 // 2 + 1) / m2
    
    valid_mask = (freqs >= 0.8) & (freqs <= 2.2)
    pows = pows[valid_mask]
    freqs = freqs[valid_mask]
    
    if len(freqs) == 0 or len(pows) == 0:
        raise ValueError("No valid frequencies found in the desired range.")
    
    idx = np.argmax(pows)
    HR_freq = freqs[idx]
    
    HR = int(np.ceil(HR_freq * 60))
    
    return HR

fps = 30 

results = [] 

for k in range(36, 43): 
    for l in range(1, 69):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\P-{k}\JAMSNet\BVP_AVI\BVP_3_{l}.csv'
            
            if not os.path.exists(bvp_csv_path):
                print(f"File not found: {bvp_csv_path}")
                continue
            
            df = pd.read_csv(bvp_csv_path)
            
            if 'Pulse' not in df.columns:
                print(f"Column 'Pulse' missing in file: {bvp_csv_path}")
                continue
            
            bvp_signal = df['Pulse'].values
            
            HR = get_HR(bvp_signal, fps)
            
            # Store results
            results.append({
                "Participant": k,
                "Clip": l,
                "HR": HR
            })
            print(f"Processed: Participant-{k}, Clip-{l}, HR={HR} BPM")
        
        except Exception as e:
            print(f"Error processing file {bvp_csv_path}: {e}")

results_df = pd.DataFrame(results)

results_df.to_csv(r'D:\UBFC\Testing\Split_Testing_JAMS\Preds_3.csv', index=False)

Processed: Participant-36, Clip-1, HR=50 BPM
Processed: Participant-36, Clip-2, HR=50 BPM
Processed: Participant-36, Clip-3, HR=50 BPM
Processed: Participant-36, Clip-4, HR=50 BPM
Processed: Participant-36, Clip-5, HR=50 BPM
Processed: Participant-36, Clip-6, HR=50 BPM
Processed: Participant-36, Clip-7, HR=50 BPM
Processed: Participant-36, Clip-8, HR=50 BPM
Processed: Participant-36, Clip-9, HR=50 BPM
Processed: Participant-36, Clip-10, HR=50 BPM
Processed: Participant-36, Clip-11, HR=50 BPM
Processed: Participant-36, Clip-12, HR=50 BPM
Processed: Participant-36, Clip-13, HR=50 BPM
Processed: Participant-36, Clip-14, HR=50 BPM
Processed: Participant-36, Clip-15, HR=50 BPM
Processed: Participant-36, Clip-16, HR=64 BPM
Processed: Participant-36, Clip-17, HR=50 BPM
Processed: Participant-36, Clip-18, HR=50 BPM
Processed: Participant-36, Clip-19, HR=50 BPM
Processed: Participant-36, Clip-20, HR=50 BPM
Processed: Participant-36, Clip-21, HR=50 BPM
Processed: Participant-36, Clip-22, HR=64 B

In [2]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(36, 43):
    for p in range(1, 17):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\Split_Testing_JAMS_Final\P-{k}\clip_{p}_BVP.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['BVP'].values
    
            lowcut = 0.8
            highcut = 2.2
            fps = 29
            order = 6
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
            
            f, Pxx = welch(filtered_bvp, fs=fps, nperseg=len(filtered_bvp), noverlap=int(0.9 * len(filtered_bvp)), nfft=2048)
            
            valid_idx = (f >= lowcut) & (f <= highcut)
            valid_f = f[valid_idx]
            valid_Pxx = Pxx[valid_idx]
            
            if valid_f.size > 0:
                max_freq = valid_f[np.argmax(valid_Pxx)]
                bpm = max_freq * 60  # Convert Hz to BPM
                bpms.append(round(bpm))
                videos.append(k)
    
                print(f'{k} = {bpm:.0f}')
        except (ValueError, OSError) as e:
            pass
    
df = pd.DataFrame({'Video': videos, 'BPM': bpms})
df.to_csv(r'D:\UBFC\Testing\Split_Testing_JAMS_Final\GT_Full.csv', index=False)

36 = 96
36 = 90
36 = 91
36 = 100
36 = 99
36 = 95
36 = 87
36 = 95
36 = 92
36 = 102
36 = 98
36 = 96
36 = 100
36 = 104
36 = 98
36 = 107
37 = 82
37 = 73
37 = 73
37 = 79
37 = 75
37 = 80
37 = 87
37 = 87
37 = 86
37 = 85
37 = 82
37 = 73
37 = 78
37 = 74
37 = 87
37 = 82
38 = 104
38 = 105
38 = 112
38 = 111
38 = 105
38 = 111
38 = 116
38 = 111
38 = 114
38 = 105
38 = 106
38 = 105
38 = 110
38 = 93
38 = 97
38 = 100
39 = 99
39 = 100
39 = 98
39 = 102
39 = 99
39 = 96
39 = 95
39 = 86
39 = 92
39 = 89
39 = 88
39 = 92
39 = 85
39 = 88
39 = 96
39 = 101
40 = 108
40 = 102
40 = 104
40 = 110
40 = 110
40 = 110
40 = 119
40 = 109
40 = 108
40 = 111
40 = 89
40 = 97
40 = 104
40 = 99
40 = 108
40 = 110
41 = 85
41 = 87
41 = 87
41 = 89
41 = 84
41 = 88
41 = 88
41 = 91
41 = 87
41 = 93
41 = 85
41 = 88
41 = 93
41 = 89
41 = 86
41 = 89
42 = 86
42 = 86
42 = 91
42 = 88
42 = 91
42 = 90
42 = 87
42 = 85
42 = 83
42 = 84
42 = 82
42 = 85
42 = 76
42 = 85
42 = 84
42 = 82


In [7]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(29, 43):
    for p in range(1, 17):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\P-{k}\PhysNet\BVP_AVI\BVP_BatchSize8_{p}.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['Pulse'].values
    
            lowcut = 0.7
            highcut = 4.0
            fps = 29
            order = 6
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
            
            # Compute Welch's method on the entire signal
            f, Pxx = welch(filtered_bvp, fs=fps, nperseg=len(filtered_bvp), noverlap=int(0.9 * len(filtered_bvp)), nfft=2048)
            
            # Filter the frequencies to the range [0.7 Hz, 4 Hz]
            valid_idx = (f >= lowcut) & (f <= highcut)
            valid_f = f[valid_idx]
            valid_Pxx = Pxx[valid_idx]
            
            if valid_f.size > 0:
                max_freq = valid_f[np.argmax(valid_Pxx)]
                bpm = max_freq * 60  # Convert Hz to BPM
                bpms.append(round(bpm))
                videos.append(k)
    
                print(f'{k} = {bpm:.0f}')
        except (ValueError, OSError) as e:
            pass
    
# Save results to a CSV file
df = pd.DataFrame({'Video': videos, 'BPM': bpms})
df.to_csv(r'D:\UBFC\Testing\Preds_Full_BatchSize8.csv', index=False)

29 = 142
29 = 67
29 = 78
29 = 110
29 = 77
29 = 139
29 = 99
29 = 97
29 = 86
29 = 88
29 = 143
29 = 77
29 = 69
29 = 116
29 = 59
29 = 68
30 = 102
30 = 82
30 = 100
30 = 84
30 = 97
30 = 134
30 = 118
30 = 74
30 = 86
30 = 144
30 = 91
30 = 82
30 = 81
30 = 70
30 = 78
31 = 59
31 = 142
31 = 143
31 = 140
31 = 143
31 = 142
31 = 141
31 = 140
31 = 144
31 = 66
31 = 144
31 = 140
31 = 141
31 = 144
31 = 103
31 = 144
32 = 101
32 = 99
32 = 94
32 = 113
32 = 110
32 = 93
32 = 95
32 = 71
32 = 83
32 = 99
32 = 115
32 = 71
32 = 87
32 = 83
32 = 50
33 = 94
33 = 78
33 = 135
33 = 102
33 = 85
33 = 77
33 = 65
33 = 76
33 = 141
33 = 78
33 = 84
33 = 80
33 = 111
33 = 82
33 = 93
34 = 60
34 = 68
34 = 52
34 = 93
34 = 107
34 = 101
34 = 89
34 = 96
34 = 90
34 = 100
34 = 87
34 = 65
34 = 74
34 = 92
34 = 75
35 = 66
35 = 67
35 = 93
35 = 60
35 = 81
35 = 99
35 = 77
35 = 99
35 = 93
35 = 71
35 = 114
35 = 117
35 = 78
35 = 111
35 = 62
36 = 65
36 = 109
36 = 61
36 = 93
36 = 87
36 = 82
36 = 71
36 = 105
36 = 92
36 = 85
36 = 100
36 = 75
36 = 79

In [23]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(29, 43):
    for p in range(1, 17):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\Split_Testing_Phys\P-{k}\clip_{p}_BVP.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['BVP'].values
    
            lowcut = 0.7
            highcut = 4.0
            fps = 29
            order = 6
            winsize = 4
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
            
            max_freqs = []
            window_size = int(winsize * fps)
            
            for start in range(0, len(filtered_bvp) - window_size + 1, 24):
                windowed_signal = filtered_bvp[start:start + window_size]
                
                f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size, nfft=2048)
                
                valid_idx = (f >= lowcut) & (f <= highcut)
                valid_f = f[valid_idx]
                valid_Pxx = Pxx[valid_idx]
                
                if valid_f.size > 0:
                    max_freq = valid_f[np.argmax(valid_Pxx)]
                    max_freqs.append(max_freq * 60)  # Convert Hz to BPM
            
            if max_freqs:
                mean_freq = np.mean(max_freqs)
                bpms.append(round(mean_freq))
                videos.append(k)
    
                print(f'{k} = {mean_freq:.0f}')
        except (ValueError, OSError) as e:
            pass
    
df = pd.DataFrame({'Video': videos, 'BPM': bpms})
df.to_csv(r'D:\UBFC\Testing\GT_4.csv', index=False)

29 = 107
29 = 94
29 = 110
29 = 111
29 = 118
29 = 116
29 = 122
29 = 119
29 = 118
29 = 122
29 = 116
29 = 116
29 = 118
29 = 116
29 = 121
29 = 123
30 = 60
30 = 69
30 = 59
30 = 63
30 = 56
30 = 59
30 = 55
30 = 51
30 = 52
30 = 53
30 = 53
30 = 55
30 = 53
30 = 51
30 = 56
31 = 108
31 = 108
31 = 107
31 = 110
31 = 107
31 = 108
31 = 107
31 = 109
31 = 106
31 = 99
31 = 103
31 = 104
31 = 106
31 = 105
31 = 109
31 = 102
32 = 81
32 = 79
32 = 80
32 = 88
32 = 83
32 = 82
32 = 79
32 = 85
32 = 74
32 = 82
32 = 81
32 = 83
32 = 80
32 = 76
32 = 85
33 = 83
33 = 82
33 = 84
33 = 88
33 = 83
33 = 85
33 = 86
33 = 81
33 = 82
33 = 86
33 = 94
33 = 83
33 = 82
33 = 82
33 = 77
34 = 80
34 = 102
34 = 100
34 = 94
34 = 92
34 = 100
34 = 121
34 = 104
34 = 102
34 = 93
34 = 97
34 = 91
34 = 97
34 = 99
34 = 99
35 = 79
35 = 77
35 = 88
35 = 86
35 = 95
35 = 90
35 = 97
35 = 93
35 = 94
35 = 97
35 = 93
35 = 96
35 = 98
35 = 93
35 = 93
36 = 97
36 = 90
36 = 91
36 = 99
36 = 100
36 = 91
36 = 89
36 = 93
36 = 93
36 = 92
36 = 90
36 = 97
36 = 99
36 

In [8]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(36, 43):
    for p in range(1, 17):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\P-{k}\JAMSNet\BVP_AVI\BVP_{p}.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['Pulse'].values
    
            lowcut = 0.8
            highcut = 2.2
            fps = 29
            order = 6
            winsize = 4
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
            
            max_freqs = []
            window_size = int(winsize * fps)
            
            for start in range(0, len(filtered_bvp) - window_size + 1, 24):
                windowed_signal = filtered_bvp[start:start + window_size]
                
                f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size, nfft=2048)
                
                valid_idx = (f >= lowcut) & (f <= highcut)
                valid_f = f[valid_idx]
                valid_Pxx = Pxx[valid_idx]
                
                if valid_f.size > 0:
                    max_freq = valid_f[np.argmax(valid_Pxx)]
                    max_freqs.append(max_freq * 60)  # Convert Hz to BPM
            
            if max_freqs:
                mean_freq = np.mean(max_freqs)
                bpms.append(round(mean_freq))
                videos.append(k)
    
                print(f'{k} = {mean_freq:.0f}')
        except (ValueError, OSError) as e:
            pass
    
df = pd.DataFrame({'Video': videos, 'BPM': bpms})
df.to_csv(r'D:\UBFC\Testing\Split_Testing_JAMS\Preds_4.csv', index=False)

36 = 93
36 = 99
36 = 105
36 = 78
36 = 95
36 = 85
36 = 97
36 = 90
36 = 106
36 = 118
36 = 96
36 = 84
36 = 118
36 = 60
36 = 101
36 = 71
37 = 100
37 = 105
37 = 111
37 = 56
37 = 83
37 = 69
37 = 81
37 = 74
37 = 94
37 = 119
37 = 73
37 = 85
37 = 82
37 = 71
37 = 74
37 = 75
38 = 103
38 = 104
38 = 113
38 = 54
38 = 101
38 = 78
38 = 92
38 = 105
38 = 94
38 = 105
38 = 98
38 = 110
38 = 106
38 = 103
38 = 93
38 = 110
39 = 96
39 = 102
39 = 103
39 = 99
39 = 100
39 = 71
39 = 99
39 = 103
39 = 102
39 = 103
39 = 102
39 = 102
39 = 93
39 = 96
39 = 94
39 = 94
40 = 108
40 = 110
40 = 104
40 = 104
40 = 105
40 = 100
40 = 105
40 = 105
40 = 106
40 = 104
40 = 104
40 = 109
40 = 113
40 = 107
40 = 107
40 = 108
41 = 92
41 = 60
41 = 68
41 = 76
41 = 92
41 = 85
41 = 107
41 = 75
41 = 97
41 = 107
41 = 71
41 = 84
41 = 84
41 = 87
41 = 85
41 = 92
42 = 55
42 = 71
42 = 56
42 = 57
42 = 68
42 = 72
42 = 55
42 = 75
42 = 55
42 = 55
42 = 54
42 = 56
42 = 58
42 = 56
42 = 55
42 = 55


In [3]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch

def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
        
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

videos = []
bpms = []

for k in range(36, 43):
    for p in range(1, 17):
        try:
            bvp_csv_path = rf'D:\UBFC\Testing\Split_Testing_JAMS_Final\P-{k}\clip_{p}_BVP.csv'
            df = pd.read_csv(bvp_csv_path)
            bvp_signal = df['BVP'].values
    
            lowcut = 0.8
            highcut = 2.2
            fps = 29
            order = 6
            winsize = 5
            
            filtered_bvp = butter_bandpass_filter(bvp_signal, lowcut, highcut, fps, order)
            
            max_freqs = []
            window_size = int(winsize * fps)  # Convert winsize in seconds to samples
            overlap_size = int(0.9 * window_size)  # 90% overlap
            
            for start in range(0, len(filtered_bvp) - window_size + 1, window_size - overlap_size):
                windowed_signal = filtered_bvp[start:start + window_size]
                
                f, Pxx = welch(windowed_signal, fs=fps, nperseg=window_size, noverlap=overlap_size, nfft=2048)
                
                valid_idx = (f >= lowcut) & (f <= highcut)
                valid_f = f[valid_idx]
                valid_Pxx = Pxx[valid_idx]
                
                if valid_f.size > 0:
                    max_freq = valid_f[np.argmax(valid_Pxx)]
                    max_freqs.append(max_freq * 60)  # Convert Hz to BPM
            
            if max_freqs:
                mean_freq = np.mean(max_freqs)
                bpms.append(round(mean_freq))
                videos.append(k)
    
                print(f'{k} = {mean_freq:.0f}')
        except (ValueError, OSError) as e:
            pass
    
# Save results to a CSV file
df = pd.DataFrame({'Video': videos, 'BPM': bpms})
df.to_csv(r'D:\UBFC\Testing\Split_Testing_JAMS_Final\GT_5.csv', index=False)

36 = 96
36 = 90
36 = 90
36 = 100
36 = 99
36 = 95
36 = 87
36 = 95
36 = 92
36 = 102
36 = 99
36 = 96
36 = 100
36 = 104
36 = 98
36 = 108
37 = 82
37 = 73
37 = 74
37 = 79
37 = 75
37 = 80
37 = 87
37 = 86
37 = 86
37 = 85
37 = 82
37 = 73
37 = 79
37 = 74
37 = 87
37 = 82
38 = 104
38 = 105
38 = 112
38 = 111
38 = 105
38 = 110
38 = 117
38 = 111
38 = 114
38 = 105
38 = 106
38 = 105
38 = 110
38 = 93
38 = 97
38 = 100
39 = 99
39 = 100
39 = 98
39 = 102
39 = 99
39 = 96
39 = 95
39 = 86
39 = 92
39 = 89
39 = 88
39 = 92
39 = 85
39 = 88
39 = 95
39 = 101
40 = 108
40 = 102
40 = 104
40 = 111
40 = 110
40 = 110
40 = 119
40 = 109
40 = 108
40 = 112
40 = 89
40 = 97
40 = 105
40 = 99
40 = 108
40 = 110
41 = 85
41 = 88
41 = 87
41 = 90
41 = 85
41 = 88
41 = 88
41 = 92
41 = 88
41 = 93
41 = 86
41 = 88
41 = 93
41 = 90
41 = 87
41 = 89
42 = 86
42 = 85
42 = 90
42 = 88
42 = 92
42 = 90
42 = 87
42 = 85
42 = 83
42 = 84
42 = 82
42 = 85
42 = 76
42 = 85
42 = 83
42 = 82
